#**REINFORCEMENT LEARNING**

---

##0.REFERENCE

https://claude.ai/share/ef8d8bcf-faa9-49ac-955c-4b19ec4c4439

##1.CONTEXT


Trading is fundamentally about making sequential decisions under uncertainty. Should you buy, sell, or hold? How much risk should you take? When should you cut losses or let winners run? Traditional approaches often treat these as separate prediction problems—forecasting returns, estimating volatility, identifying regime shifts. But reinforcement learning (RL) offers a fundamentally different perspective: it optimizes the entire decision process, not just individual predictions.

This chapter introduces you to RL for trading by building a complete, governance-ready system from the ground up. Unlike typical RL tutorials that rely on opaque libraries and toy examples, we implement everything transparently using only NumPy and Python's standard library. You'll see exactly how the algorithms work, why certain design choices matter, and where the pitfalls lie.

**Why Reinforcement Learning?**

Think of a discretionary trader learning their craft. They don't just study price charts—they learn through experience. Each trade teaches them something: this pattern worked in high volatility, that strategy hemorrhaged transaction costs, this risk limit saved them from ruin. RL formalizes this learning process. It treats trading as a Markov Decision Process where states (market conditions, portfolio positions) lead to actions (trades) that generate rewards (profits minus costs and penalties). The goal is to learn a policy—a decision rule—that maximizes long-run rewards while respecting real-world constraints.

The power of RL is that it naturally incorporates everything that matters: transaction costs that erode returns, position limits that prevent catastrophic losses, drawdown penalties that capture risk aversion, and the delayed consequences of today's decisions. Unlike supervised learning models that predict "what will happen," RL optimizes "what should I do."

**The Reality Check: Offline RL and Backtesting**

But here's the critical insight this chapter emphasizes: RL for trading is hard, and most naive approaches fail spectacularly. The core challenge is that we can't experiment freely with real money. We must learn offline from historical data, then deploy in live markets—a setting where textbook RL algorithms often produce dangerously overconfident policies.

This notebook demonstrates conservative offline RL through two stages. First, behavior cloning learns from a sensible expert policy (a simple trend-following strategy). This creates a stable baseline that captures proven trading logic. Second, conservative policy improvement makes small, cautious steps away from this baseline—improving performance while maintaining a safety margin. This mirrors how experienced traders evolve their strategies: start with what works, make incremental changes, stress-test relentlessly.

We also confront the measurement problem head-on. Off-policy evaluation (OPE)—estimating how a new policy would perform using old data—can be wildly misleading. Small differences between the behavior policy (which generated the data) and the target policy (which we're evaluating) can cause importance sampling weights to explode, yielding useless estimates. The notebook includes a live demonstration of this phenomenon, showing why walk-forward backtests remain essential despite their limitations.

**Governance as a First Principle**

Finally, this chapter treats governance not as an afterthought but as a design requirement. Every decision is logged, every dataset fingerprinted, every model versioned. We generate reproducible bundles with configuration hashes, causality proofs, stress test results, and audit trails. This isn't bureaucracy—it's survival. In production trading, you must be able to explain why your system made each trade, prove it didn't peek at future data, and demonstrate robustness to cost shocks and market regime changes.

By the end, you'll understand RL not as a magic bullet, but as a powerful tool that requires deep respect for its assumptions, careful engineering of its components, and relentless skepticism about its outputs.

##2.LIBRARIES AND ENVIRONMENT

In [1]:

# Cell 2 — Determinism + project paths + hashing utilities
import numpy as np
import json
import os
import hashlib
from datetime import datetime
import math
import random
from collections import defaultdict
import itertools

# Set master seed for reproducibility
MASTER_SEED = 42
np.random.seed(MASTER_SEED)
random.seed(MASTER_SEED)

# Derive sub-seeds deterministically
def derive_seed(base_seed, label):
    """Derive a sub-seed from base seed and a label."""
    h = hashlib.md5(f"{base_seed}_{label}".encode()).digest()
    return int.from_bytes(h[:4], 'big') % (2**31)

SEED_DATA = derive_seed(MASTER_SEED, "data")
SEED_TRAIN = derive_seed(MASTER_SEED, "train")
SEED_EVAL = derive_seed(MASTER_SEED, "eval")

# Create run folder structure
run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
BASE_PATH = f"/content/ch19_runs/{run_id}"
PATHS = {
    'base': BASE_PATH,
    'artifacts': f"{BASE_PATH}/artifacts",
    'plots': f"{BASE_PATH}/plots",
    'logs': f"{BASE_PATH}/logs",
    'policy': f"{BASE_PATH}/policy",
    'data': f"{BASE_PATH}/data"
}

for path in PATHS.values():
    os.makedirs(path, exist_ok=True)

print(f"[INIT] Run ID: {run_id}")
print(f"[INIT] Base path: {BASE_PATH}")

# Hashing utilities for governance
def stable_hash_dict(d):
    """Compute stable hash of dictionary (sorted JSON)."""
    s = json.dumps(d, sort_keys=True, indent=None)
    return hashlib.sha256(s.encode()).hexdigest()

def file_hash(filepath):
    """Compute SHA-256 hash of file."""
    h = hashlib.sha256()
    with open(filepath, 'rb') as f:
        for chunk in iter(lambda: f.read(8192), b''):
            h.update(chunk)
    return h.hexdigest()

def array_fingerprint(arr):
    """Compute fingerprint of numpy array."""
    return hashlib.sha256(arr.tobytes()).hexdigest()[:16]

print("[INIT] Hashing utilities ready.")

[INIT] Run ID: 20251229_173257
[INIT] Base path: /content/ch19_runs/20251229_173257
[INIT] Hashing utilities ready.


##3.CONFIG REGISTRY

###3.1.OVERVIEW



This section establishes the governance backbone of our RL trading system by creating a
comprehensive configuration registry and run manifest. Think of this as the "birth certificate"
for our experiment—every parameter, assumption, and design choice is documented, hashed, and
saved before any computation begins.

**Why Configuration Management Matters**

In production trading systems, reproducibility isn't optional—it's existential. When a strategy
loses money, you need to know exactly what it was doing and why. When regulators ask questions,
you need auditable records. When you want to improve a model, you need to know precisely what
the baseline was. Ad-hoc parameter choices scattered through code make all of this impossible.
This section creates a single source of truth: a JSON configuration file that captures every
consequential decision.

**The Configuration Dictionary**

Our config registry organizes parameters into logical groups:

- **Data Generation Parameters**: How we create synthetic markets—number of timesteps, regime
switching dynamics, volatility levels by regime, drift rates. This defines our "universe" for
training and testing. For synthetic data, we explicitly record the random seed to ensure
perfect reproducibility.

- **Execution Model**: The timing convention (decisions at t, fills at t+1), step frequency
(daily in our case), and slippage assumptions. These choices profoundly affect results—a
strategy that looks profitable with instant execution might fail with realistic delays.

- **Cost Model**: Transaction fees (5 bps), bid-ask spread (2 bps), and market impact
coefficient. These are often the difference between paper profits and real losses. We make
them explicit and auditable.

- **Constraint Set**: Position limits (±1.0), leverage caps, and turnover restrictions. Real
trading operates under risk limits—our RL agent must learn to respect them. These aren't soft
preferences; they're hard boundaries enforced in the environment.

- **Reward Function Coefficients**: How much we penalize risk (variance), drawdowns, and
excessive turnover. These encode our risk preferences and implicitly define "good" vs "bad"
trading. Changing these coefficients fundamentally changes what the RL agent optimizes for.

- **Training Hyperparameters**: Learning rates, batch sizes, number of epochs for behavior
cloning, steps for conservative policy improvement. These affect convergence and stability.

- **Evaluation Protocol**: Walk-forward window sizes (800 steps training, 200 steps testing),
step length (how far forward we move between folds), minimum warmup period. This defines our
backtesting methodology and controls data leakage risk.

- **Stress Test Grid**: Cost inflation factors, latency shifts, liquidity shocks, regime-based
slicing. We pre-specify our stress scenarios so they can't be cherry-picked later.

**The Configuration Hash**

After building the configuration dictionary, we compute a cryptographic hash (SHA-256) of its
contents. This creates a unique fingerprint: if any parameter changes—even by a single digit—
the hash changes completely. This hash becomes part of all downstream artifacts, allowing us
to trace every result back to its exact configuration. It prevents the common failure mode
where someone tweaks a parameter, gets different results, but can't remember what they changed.

**The Run Manifest**

While the config specifies "what we're testing," the run manifest records "when and how we
tested it." It captures:

- Unique run ID and timestamp
- Master random seed and all derived sub-seeds
- Config hash (linking to the configuration)
- Code hash placeholder (filled in later to version the code itself)
- Environment version string
- Status flag (running/complete/failed)

This manifest acts as metadata that connects configuration, code, data, and results into a
coherent audit trail.

**Why This Matters in Practice**

When a trading strategy fails in production, the typical post-mortem involves frantic searches
through old notebooks, trying to remember what parameters were used. Was the transaction cost
5 bps or 10 bps? Did we include the drawdown penalty? Which random seed generated that
particular dataset?

This section eliminates that chaos. Every run is self-documenting. You can reproduce any
result months later by loading the config file and using the recorded seeds. You can compare
two runs by comparing their config hashes. You can audit regulatory compliance by showing that
your risk limits were enforced from day one, not added retroactively.

This is governance-first design: rather than bolting on documentation after getting results,
we make documentation a prerequisite for running the experiment. The few minutes spent setting
this up saves hours—or careers—later.

###3.2.CODE AND IMPLEMENTATION

In [2]:

# Cell 3 — Config registry + run manifest skeleton
CONFIG = {
    # Data generator parameters
    'data': {
        'T': 2000,  # Total timesteps
        'n_regimes': 2,
        'regime_names': ['low_vol', 'high_vol'],
        'transition_matrix': [[0.98, 0.02], [0.05, 0.95]],
        'vol_by_regime': [0.01, 0.03],  # Daily vol by regime
        'drift_by_regime': [0.0001, 0.0002],  # Daily drift
        'initial_price': 100.0,
        'seed': SEED_DATA
    },

    # Decision cadence and execution
    'execution': {
        'decision_step': 1,  # Decide every 1 day
        'fill_timing': 'next_step',  # Execute at t+1
        'slippage_model': 'proportional'
    },

    # Cost model
    'costs': {
        'fee_bps': 5.0,  # 5 bps transaction fee
        'spread_bps': 2.0,  # 2 bps spread
        'impact_coeff': 0.1,  # Price impact coefficient
        'liquidity_proxy': True
    },

    # Action constraints
    'constraints': {
        'position_bounds': [-1.0, 1.0],  # Position limits
        'leverage_cap': 1.0,
        'turnover_cap_per_step': 0.5  # Max 50% turnover per step
    },

    # Reward coefficients
    'reward': {
        'risk_penalty': 0.5,  # Penalty for variance
        'drawdown_penalty': 1.0,  # Penalty for drawdown
        'turnover_penalty': 0.01  # Penalty for turnover
    },

    # Training parameters
    'training': {
        'bc_epochs': 100,
        'bc_lr': 0.01,
        'bc_batch_size': 32,
        'cpi_steps': 5,  # Reduced for efficiency
        'cpi_deviation_penalty': 1.0,
        'cpi_lr': 0.005,
        'seed_train': SEED_TRAIN,
        'seed_eval': SEED_EVAL
    },

    # Walk-forward splits
    'evaluation': {
        'train_len': 800,
        'test_len': 200,
        'step_len': 200,  # Move forward by 200 steps
        'min_train_start': 100  # Require at least 100 warmup steps
    },

    # Stress test grid
    'stress_tests': {
        'cost_inflation': [1.0, 1.5, 2.0],
        'latency_shift': [0, 1],  # 0 = t+1, 1 = t+2
        'liquidity_shock': [1.0, 0.5],  # Reduce liquidity
        'regime_slice': True  # Evaluate by regime
    },

    # Baselines to run
    'baselines': ['cash', 'buy_hold', 'trend', 'myopic', 'imitation']
}

# Save config
config_path = f"{PATHS['artifacts']}/config.json"
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=2)

config_hash = stable_hash_dict(CONFIG)
print(f"[CONFIG] Config hash: {config_hash}")

# Create run manifest
run_manifest = {
    'run_id': run_id,
    'timestamp_start': datetime.now().isoformat(),
    'master_seed': MASTER_SEED,
    'config_hash': config_hash,
    'code_hash': 'TBD',  # Will compute later
    'environment_version': 'ch19_v1.0',
    'status': 'running'
}

manifest_path = f"{PATHS['artifacts']}/run_manifest.json"
with open(manifest_path, 'w') as f:
    json.dump(run_manifest, f, indent=2)

print("[CONFIG] Config and manifest saved.")



[CONFIG] Config hash: 01ddc987772ceb9a59fb9bd082ea3212ac9d300ddaabe1048e02d1a3c606aa5b
[CONFIG] Config and manifest saved.


##4.SYNTHETIC MARKET GENERATOR

###4.1.OVERVIEW



This section creates the synthetic market environment where our RL agent will learn to trade.
Rather than downloading real market data, we generate a controlled artificial market with
known properties. This pedagogical choice gives us ground truth, perfect reproducibility, and
the ability to stress-test under specific conditions—advantages that real data cannot provide.

**Why Synthetic Data First**

Real market data is messy, non-stationary, and comes with survivorship bias, look-ahead bias,
and corporate actions that complicate analysis. When learning RL concepts, these complexities
obscure the fundamental mechanisms. Synthetic data lets us isolate what we're studying: can
the RL agent learn to trade profitably under costs and constraints when the market exhibits
regime-switching behavior? We know the answer should be "yes" because we designed the regimes
to be detectable. If the agent fails here, it will certainly fail on real data.

Moreover, synthetic data is perfectly reproducible. By setting a random seed, we generate
identical price series every time. This eliminates a major source of confusion in RL research:
did performance change because we improved the algorithm, or because we got lucky with a
different market sample?

**The Market Model: Regime-Switching Dynamics**

Our synthetic market features two regimes—low volatility and high volatility—that follow a
Markov chain. Think of these as "calm markets" and "turbulent markets." The system starts in
one regime and probabilistically transitions between them according to a transition matrix.

- **Regime 0 (Low Volatility)**: Daily volatility of 1%, small positive drift. This represents
stable market conditions where trends persist and risk is moderate.

- **Regime 1 (High Volatility)**: Daily volatility of 3%, slightly higher drift. This captures
turbulent periods where prices swing wildly and risk management becomes critical.

- **Transition Probabilities**: High probability of staying in the current regime (98% for low
vol, 95% for high vol), low probability of switching. This creates realistic regime persistence—
markets don't flip between calm and chaotic every day.

The regime sequence is generated first using the Markov chain. Then, for each timestep, returns
are drawn from a normal distribution with mean and variance determined by the current regime.
This creates heteroskedastic returns (volatility clustering) without requiring complex GARCH
models.

**From Returns to Prices**

Given the return series, we construct prices through simple compounding: each price equals the
previous price multiplied by (1 + return). We start at an initial price of 100, making
percentage changes easy to interpret. This price series is what a human trader would see on a
chart, while the returns are what drive P&L.

**The Liquidity Proxy**

Real markets aren't perfectly liquid—larger trades incur greater market impact. We model this
by creating a liquidity proxy inversely related to volatility: high-volatility regimes have
lower liquidity (trades are more expensive), while low-volatility regimes have higher liquidity.
We add noise to prevent the RL agent from perfectly inferring regime from liquidity alone. This
liquidity proxy will feed into our cost model, making transaction costs state-dependent and
realistic.

**Data Governance and Fingerprinting**

After generating the data, we immediately save it to disk in NumPy's compressed format (.npz).
This preserves the arrays efficiently without requiring pandas. More importantly, we compute
and save a data fingerprint—a hash-based signature that uniquely identifies this dataset.

The fingerprint JSON records:

- Instrument identifier (synthetic_1)
- Frequency (daily)
- Number of timesteps
- Hash of returns array (16-character hex string)
- Hash of prices array
- Missingness rate (0% for synthetic data)
- Corporate actions (none)
- Random seed used for generation

This fingerprint serves multiple purposes. First, it lets us verify data integrity—if we
reload the data later, we can recompute the hash to confirm nothing corrupted. Second, it
provides traceability: every model trained on this data can reference this fingerprint,
creating an audit trail from results back to exact data sources.

**Visualization for Sanity Checks**

The section concludes by plotting three time series: prices, returns, and regime indicators.
These plots aren't just pretty pictures—they're essential sanity checks. We verify that:

- Prices follow reasonable trajectories (no sudden jumps to infinity)
- Returns exhibit regime-dependent volatility clustering (visible heteroskedasticity)
- Regime switches occur at realistic frequencies (not too fast, not too slow)

If something looks wrong in these plots, we catch it now, before wasting compute time training
on broken data.

**The No-Pandas Constraint**

Notice we generate everything using NumPy arrays and explicit loops. No pandas DataFrames, no
rolling windows, no datetime indices. This constraint may seem arbitrary, but it serves
pedagogical and practical purposes. It forces us to think clearly about time indices, makes
the code portable to production systems where pandas may be banned, and eliminates a common
source of subtle bugs (timezone handling, forward-filling, implicit reindexing). Every
operation is explicit, auditable, and unambiguous.

###4.2.CODE AND IMPLEMENTATION

In [3]:

def generate_synthetic_market(config):
    """
    Generate synthetic market data with regime switching.
    Returns: dict with 'returns', 'prices', 'regimes', 'liquidity'
    """
    np.random.seed(config['seed'])

    T = config['T']
    n_regimes = config['n_regimes']
    trans_matrix = np.array(config['transition_matrix'])
    vols = np.array(config['vol_by_regime'])
    drifts = np.array(config['drift_by_regime'])
    initial_price = config['initial_price']

    # Generate regime sequence using Markov chain
    regimes = np.zeros(T, dtype=int)
    regimes[0] = 0  # Start in regime 0

    for t in range(1, T):
        # Transition probabilities from current regime
        probs = trans_matrix[regimes[t-1]]
        regimes[t] = np.random.choice(n_regimes, p=probs)

    # Generate returns based on regime
    returns = np.zeros(T)
    for t in range(T):
        regime = regimes[t]
        returns[t] = drifts[regime] + vols[regime] * np.random.randn()

    # Generate prices
    prices = np.zeros(T)
    prices[0] = initial_price
    for t in range(1, T):
        prices[t] = prices[t-1] * (1 + returns[t])

    # Generate liquidity proxy (inverse of volatility + noise)
    liquidity = np.zeros(T)
    for t in range(T):
        base_liq = 1.0 / (vols[regimes[t]] + 0.001)
        liquidity[t] = base_liq * (1 + 0.1 * np.random.randn())
        liquidity[t] = max(liquidity[t], 0.1)  # Floor

    return {
        'returns': returns,
        'prices': prices,
        'regimes': regimes,
        'liquidity': liquidity,
        'T': T
    }

# Generate data
print("[DATA] Generating synthetic market data...")
market_data = generate_synthetic_market(CONFIG['data'])

# Save dataset
dataset_path = f"{PATHS['data']}/synthetic_market.npz"
np.savez(dataset_path,
         returns=market_data['returns'],
         prices=market_data['prices'],
         regimes=market_data['regimes'],
         liquidity=market_data['liquidity'])

# Compute data fingerprint
data_fingerprint = {
    'instrument': 'synthetic_1',
    'frequency': 'daily',
    'span': market_data['T'],
    'returns_fingerprint': array_fingerprint(market_data['returns']),
    'prices_fingerprint': array_fingerprint(market_data['prices']),
    'missingness': 0.0,
    'corporate_actions': 'none',
    'seed': CONFIG['data']['seed']
}

fingerprint_path = f"{PATHS['data']}/data_fingerprint.json"
with open(fingerprint_path, 'w') as f:
    json.dump(data_fingerprint, f, indent=2)

print(f"[DATA] Generated {market_data['T']} timesteps.")
print(f"[DATA] Returns fingerprint: {data_fingerprint['returns_fingerprint']}")

# Plot market data
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 1, figsize=(12, 8))

# Prices
axes[0].plot(market_data['prices'])
axes[0].set_title('Synthetic Prices')
axes[0].set_ylabel('Price')
axes[0].grid(True, alpha=0.3)

# Returns
axes[1].plot(market_data['returns'])
axes[1].set_title('Returns')
axes[1].set_ylabel('Return')
axes[1].grid(True, alpha=0.3)

# Regimes
axes[2].plot(market_data['regimes'])
axes[2].set_title('Regime (0=Low Vol, 1=High Vol)')
axes[2].set_ylabel('Regime')
axes[2].set_xlabel('Time')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{PATHS['plots']}/market_data.png", dpi=100)
plt.close()

print("[DATA] Market data plots saved.")


[DATA] Generating synthetic market data...
[DATA] Generated 2000 timesteps.
[DATA] Returns fingerprint: b5f7954e2b6ee58b
[DATA] Market data plots saved.


##5.COST MODEL REGISTRY

###5.1.OVERVIEW



This section tackles one of the most critical—and most commonly botched—aspects of algorithmic
trading: realistic transaction costs. Many academic papers and online tutorials assume frictionless
markets where trades execute instantly at mid-price with no fees. In reality, every trade incurs
multiple costs that can transform paper profits into actual losses. This section builds an explicit,
auditable cost model and registers it as a governed artifact.

**The Three Components of Trading Costs**

Our cost model decomposes transaction costs into three distinct mechanisms:

- **Fixed Fees**: A flat 5 basis points (0.05%) per trade, representing exchange fees, clearing
costs, and broker commissions. This is the simplest component—it applies uniformly regardless of
market conditions or trade size. For a $10,000 trade, you pay $5 in fees.

- **Bid-Ask Spread**: An additional 2 basis points capturing the cost of crossing the spread.
When you buy, you pay the ask price; when you sell, you receive the bid price. The difference is
the spread, and it represents compensation to market makers for providing liquidity. This cost is
unavoidable in real markets.

- **Market Impact**: The most sophisticated component, proportional to trade size and inversely
proportional to liquidity. Large trades move prices against you—buying pushes prices up, selling
pushes them down. Our impact model scales with the absolute trade size and divides by a liquidity
proxy, so the same $10,000 trade costs more in illiquid (high volatility) conditions than in
liquid (low volatility) conditions.

The total cost formula becomes: **(fees + spread) + (impact_coefficient × |trade_size| / liquidity)**,
expressed as a fraction of the notional trade amount. For a trade of 0.5 units (50% position change)
in normal liquidity conditions, you might pay 10-15 basis points total—small individually, but
devastating for high-frequency strategies.

**Why Cost Modeling Matters**

Transaction costs are the graveyard of trading strategies. A strategy that generates 20% annual
returns in simulation might produce -5% after costs in reality. High-turnover strategies are
particularly vulnerable: if you trade daily and each round-trip costs 20 bps, you're burning 50%
annually before making a single dollar in market returns.

The RL agent must learn to trade profitably *net of costs*. By incorporating costs directly into
the reward function, the agent naturally learns cost-aware behavior: it avoids excessive turnover,
times trades to coincide with higher liquidity, and only takes positions when expected returns
justify the transaction costs. This is fundamentally different from training on gross returns and
hoping the strategy remains profitable after costs.

**Liquidity-Dependent Costs: The Realism Factor**

The liquidity proxy makes our cost model state-dependent and realistic. In calm market conditions
(low volatility regime), liquidity is high and impact costs are modest. During market turbulence
(high volatility regime), liquidity evaporates and the same trade size incurs much larger impact.
This creates a natural risk management incentive: the RL agent should trade less aggressively
precisely when costs are highest.

This feature captures a key aspect of real market microstructure that simpler models miss. It also
creates interesting strategic trade-offs: should you exit a losing position immediately (incurring
high costs in volatile conditions) or wait for calmer markets (risking further losses but paying
lower costs)?

**The Cost Model Registry as Governance**

Rather than hiding the cost function in the code, we create an explicit registry document that
lives alongside our results. This JSON file records:

- Model version identifier (v1.0)
- Mathematical formula in plain text
- All parameter values (fee_bps, spread_bps, impact_coeff)
- Units and interpretation
- Sensitivity grid for stress testing

This registry serves multiple governance functions. First, it makes our assumptions transparent—
anyone auditing our results can see exactly what cost model we used. Second, it enables systematic
sensitivity analysis: we pre-define cost inflation factors (1.0×, 1.5×, 2.0×) that we'll use in
stress tests. Third, it creates version control: if we later improve the cost model, we can track
which results used which version.

**The Execution Timing Convention**

Alongside costs, this section establishes our execution timing convention: decisions made at time
*t* execute at time *t+1* using the *t+1* price. This one-step delay is critical for causality—it
ensures the agent cannot cheat by "trading on" information it shouldn't have. When you decide to
buy at 3 PM, you don't get the 3 PM price; you get the price when your order fills, which might be
seconds, minutes, or hours later.

This delay also naturally incorporates latency and eliminates look-ahead bias. The RL agent can
only use information available *before* making the decision. The realized return and executed price
come *after* the decision, creating proper causal ordering.

**Practical Impact on Strategy Design**

By making costs explicit, auditable, and realistic, this section fundamentally shapes what the RL
agent can learn. Strategies that work in frictionless markets—like daily rebalancing to exact
portfolio targets—become unprofitable once costs are included. The agent must discover cost-effective
approaches: longer holding periods, tolerance bands around targets, and liquidity-aware execution.

This is governance in action: not just documenting what we did, but designing the system so that
what we learn is actually deployable in reality.

###5.2.CODE AND IMPLEMENTATION

In [4]:

# Cell 5 — Cost model registry + execution model
def compute_trading_cost(trade_size, liquidity, config):
    """
    Compute trading cost for a given trade.
    Cost = fees + spread + impact

    trade_size: absolute value of position change
    liquidity: liquidity proxy (higher = more liquid)
    """
    fee_bps = config['costs']['fee_bps']
    spread_bps = config['costs']['spread_bps']
    impact_coeff = config['costs']['impact_coeff']

    # Base cost in bps
    base_cost = fee_bps + spread_bps

    # Impact cost (proportional to trade size and inverse of liquidity)
    if config['costs']['liquidity_proxy']:
        impact_bps = impact_coeff * abs(trade_size) * 1000 / liquidity
    else:
        impact_bps = impact_coeff * abs(trade_size) * 1000

    total_cost_bps = base_cost + impact_bps
    total_cost_fraction = total_cost_bps / 10000.0

    return total_cost_fraction

# Create cost model registry
cost_model_registry = {
    'model_version': 'v1.0',
    'formula': 'cost = fees + spread + impact * |trade| / liquidity',
    'parameters': CONFIG['costs'],
    'units': 'fraction of trade notional',
    'notes': 'Impact cost is proportional to trade size and inversely proportional to liquidity proxy.',
    'sensitivity_grid': {
        'fee_inflation': [1.0, 1.5, 2.0],
        'spread_inflation': [1.0, 1.5, 2.0],
        'impact_inflation': [1.0, 2.0, 3.0]
    }
}

cost_registry_path = f"{PATHS['artifacts']}/cost_model_registry.json"
with open(cost_registry_path, 'w') as f:
    json.dump(cost_model_registry, f, indent=2)

print("[COSTS] Cost model registry saved.")


[COSTS] Cost model registry saved.


##6.TRADING ENVIRONMENT

###6.1.OVERVIEW



This section builds the core RL infrastructure: the trading environment where our agent will learn.
Think of this as constructing a realistic but controlled simulation of market interaction. The
environment defines what the agent can observe (state), what actions it can take, what rewards it
receives, and critically, ensures that information flow respects real-world causality—no peeking
into the future.

**The Environment as MDP Formalization**

In reinforcement learning terminology, we're creating a Markov Decision Process (MDP)—a mathematical
framework where an agent observes states, takes actions, receives rewards, and transitions to new
states. For trading, this means:

- **State**: Everything the agent knows at time *t* before making a decision
- **Action**: The target position the agent chooses (between -1 and +1)
- **Reward**: Profit/loss minus costs minus penalties, realized at time *t+1*
- **Transition**: How the market and portfolio evolve from *t* to *t+1*

The environment class implements this formalization through two key methods: `reset()` initializes
an episode (a trading sequence from start to end), and `step()` executes one action and advances
time.

**State Construction: What Can the Agent See?**

The state vector at time *t* contains only causally admissible information—data available before
the trading decision. Our state includes:

- **Lagged Returns (20 features)**: Past returns from *t-1*, *t-2*, ..., *t-20*. These capture
recent price momentum and mean reversion patterns without looking ahead.

- **Rolling Volatility (1 feature)**: Standard deviation of returns over the past 20 periods,
computed causally. This gives the agent a real-time risk estimate.

- **Regime Probabilities (2 features)**: Filtered estimates of being in each regime based on recent
volatility. Critically, this is a *filter* not a *smoother*—it uses only past data, not future data
that would be unavailable in real-time trading.

- **Portfolio State (4 features)**: Current position, equity value, drawdown from peak, and
cumulative turnover. These let the agent track its own risk exposure and constraint utilization.

This gives us a 27-dimensional state vector. Notice what's *not* included: no forward-looking
information, no smoothed regime estimates using future data, no next-period returns. Everything is
strictly backward-looking or contemporaneous.

**Action Space: Constrained Decision-Making**

The agent chooses a target position between -1.0 (fully short) and +1.0 (fully long), with 0
representing cash. But the environment doesn't blindly execute whatever the agent requests—it
projects actions through a constraint checker:

- **Position Bounds**: Hard limits at ±1.0
- **Leverage Cap**: Cannot exceed 1× leverage
- **Turnover Cap**: Cannot change position by more than 0.5 per step

If the agent tries to violate these constraints, the environment automatically clips the action to
the feasible set. This mimics real trading where risk systems override decisions that breach limits.
The agent must learn to work within these constraints, not fight against them.

**Reward Function: Optimizing What Matters**

The reward combines multiple objectives into a single scalar signal:

- **Realized P&L**: Position × return × equity—the actual money made or lost
- **Transaction Costs**: Fees + spread + impact, subtracted from P&L
- **Risk Penalty**: Coefficient × rolling volatility squared, discouraging excessive variance
- **Drawdown Penalty**: Coefficient × current drawdown from peak, punishing large losses
- **Turnover Penalty**: Coefficient × trade size, discouraging excessive trading

This multi-objective reward encodes our trading philosophy: we want profits, but not at the cost
of unbounded risk, catastrophic drawdowns, or churning the portfolio for no reason. The coefficients
(defined in Cell 3's config) determine the trade-offs between these competing goals.

**Execution Timing: The Critical One-Step Delay**

When the agent chooses action *a_t* at time *t*, execution happens at time *t+1*:

1. Decision at *t* using state *s_t* (built from data up to *t*)
2. Compute trade = *a_t* - current_position
3. Advance to *t+1*
4. Execute trade at price/return observed at *t+1*
5. Compute costs using *t+1* liquidity
6. Update portfolio and compute reward
7. Return new state *s_{t+1}*

This one-step delay is non-negotiable for causality. It ensures the agent cannot trade on information
from the future. The price at which you execute and the return you earn are unknowable at decision
time—they're stochastic outcomes that materialize after you commit to the action.

**Causality Assertions: Fail-Fast Design**

The environment includes explicit assertion statements that verify causality constraints:

- State construction only accesses data with index ≤ current_idx
- Regime filtering uses no future information
- Rewards use returns from *t+1*, not *t*
- Rolling statistics computed with causal loops, not vectorized operations that might peek ahead

If any assertion fails, the code raises an error immediately rather than silently producing leakage.
This "fail-fast" philosophy catches bugs during development rather than allowing them to contaminate
results.

**The Environment Specification Document**

After building the environment, we save a complete specification as JSON:

- List of state variables and their dimensions
- Action space type (continuous) and bounds
- Reward formula in plain text
- Constraint set
- Timing conventions with explicit execution lag
- Causality guarantee statement

This document becomes part of our governance bundle. Anyone reviewing our RL system can read this
spec and understand exactly how the agent interacts with the market, what information it has access
to, and what constraints it must respect.

**Why This Matters**

Environment design determines what your RL agent can possibly learn. A poorly designed environment—
with look-ahead bias, unrealistic execution, or misaligned rewards—will produce an agent that looks
great in simulation but fails catastrophically in production. This section invests significant effort
in getting the environment right because everything downstream depends on it. Garbage environment
equals garbage policy, regardless of how sophisticated your RL algorithm is.

###6.2.CODE AND ENVIRONMENT

In [5]:
# Cell 6 — Trading environment (auditable environment spec)
class TradingEnvironment:
    """
    Minimal trading environment for RL.

    State: admissible features at time t (no future peeking)
    Action: target position in [pos_min, pos_max]
    Reward: net P&L minus costs minus penalties

    CAUSALITY GUARANTEE: All features use data <= t only.
    """

    def __init__(self, market_data, config):
        self.returns = market_data['returns']
        self.prices = market_data['prices']
        self.regimes = market_data['regimes']
        self.liquidity = market_data['liquidity']
        self.T = len(self.returns)
        self.config = config

        # Action constraints
        self.pos_min, self.pos_max = config['constraints']['position_bounds']
        self.leverage_cap = config['constraints']['leverage_cap']
        self.turnover_cap = config['constraints']['turnover_cap_per_step']

        # State configuration
        self.lookback = 20  # Number of lagged returns
        self.vol_window = 20  # Window for rolling vol

        # Episode state
        self.reset(0, self.T)

    def reset(self, start_idx, end_idx):
        """Reset environment for episode from start_idx to end_idx."""
        # Ensure indices are integers
        self.start_idx = int(start_idx)
        self.end_idx = int(end_idx)
        self.current_idx = self.start_idx

        # Portfolio state
        self.position = 0.0
        self.cash = 1.0  # Start with 1 unit of capital
        self.equity = 1.0
        self.entry_price = self.prices[self.start_idx]
        self.peak_equity = 1.0
        self.cumulative_turnover = 0.0

        return self._get_state()

    def _get_state(self):
        """
        Construct admissible state at current time.
        CAUSALITY: Only uses data up to current_idx.
        """
        t = self.current_idx

        # Lagged returns (lookback periods)
        lagged_returns = np.zeros(self.lookback)
        for i in range(self.lookback):
            idx = t - i - 1
            if idx >= 0:
                lagged_returns[i] = self.returns[idx]

        # Rolling volatility (computed causally)
        rolling_vol = self._compute_rolling_vol(t)

        # Regime probability estimate (FILTERED, not smoothed)
        regime_prob = self._estimate_regime_prob(t)

        # Portfolio state
        portfolio_state = np.array([
            self.position,
            self.equity,
            (self.equity - self.peak_equity) / self.peak_equity,  # Drawdown
            self.cumulative_turnover
        ])

        # Combine all features
        state = np.concatenate([
            lagged_returns,
            [rolling_vol],
            regime_prob,
            portfolio_state
        ])

        return state

    def _compute_rolling_vol(self, t):
        """Compute rolling volatility up to time t (causal)."""
        window = self.vol_window
        start = max(0, t - window)
        if t - start < 2:
            return 0.01  # Default vol
        returns_window = self.returns[start:t]
        return np.std(returns_window)

    def _estimate_regime_prob(self, t):
        """
        Estimate regime probability using simple filtered approach.
        Returns P(regime=k | data up to t) for each regime.
        """
        # Simple heuristic: use recent volatility to estimate regime
        window = 10
        start = max(0, t - window)
        if t - start < 2:
            return np.array([0.5, 0.5])  # Uniform prior

        recent_vol = np.std(self.returns[start:t])
        vols = np.array(self.config['data']['vol_by_regime'])

        # Likelihood of each regime given observed vol
        # P(vol | regime) ~ exp(-0.5 * ((vol - regime_vol) / regime_vol)^2)
        likelihoods = np.exp(-0.5 * ((recent_vol - vols) / (vols + 1e-6))**2)
        probs = likelihoods / (likelihoods.sum() + 1e-6)

        return probs

    def step(self, action):
        """
        Execute action and advance one timestep.

        Timing:
        - Decision at t using state_t
        - Execution at t+1 using price[t+1], return[t+1]
        - Reward computed for transition t -> t+1
        """
        t = self.current_idx

        # Project action to satisfy constraints
        action = self._project_action(action)

        # Compute trade
        trade = action - self.position
        trade_size = abs(trade)

        # Check if we can advance
        if t + 1 >= self.end_idx:
            done = True
            next_state = self._get_state()
            reward = 0.0
            info = {'constraint_violation': False}
            return next_state, reward, done, info

        # Execute at t+1
        self.current_idx = t + 1
        realized_return = self.returns[self.current_idx]
        liquidity = self.liquidity[self.current_idx]

        # Compute cost
        cost = compute_trading_cost(trade_size, liquidity, self.config)
        cost_amount = cost * trade_size * self.equity

        # Update portfolio
        pnl = self.position * realized_return * self.equity
        self.equity = self.equity + pnl - cost_amount
        self.position = action
        self.cumulative_turnover += trade_size

        # Update peak for drawdown
        self.peak_equity = max(self.peak_equity, self.equity)

        # Compute reward
        reward = self._compute_reward(pnl, cost_amount, trade_size)

        # Check for constraint violations
        constraint_violation = (abs(self.position) > self.leverage_cap or
                                trade_size > self.turnover_cap)

        done = (self.current_idx + 1 >= self.end_idx)
        next_state = self._get_state()

        info = {
            'pnl': pnl,
            'cost': cost_amount,
            'trade_size': trade_size,
            'constraint_violation': constraint_violation,
            'equity': self.equity,
            'position': self.position
        }

        return next_state, reward, done, info

    def _project_action(self, action):
        """Project action to satisfy constraints."""
        # Position bounds
        action = np.clip(action, self.pos_min, self.pos_max)

        # Leverage cap
        action = np.clip(action, -self.leverage_cap, self.leverage_cap)

        # Turnover cap
        trade = action - self.position
        if abs(trade) > self.turnover_cap:
            trade = np.sign(trade) * self.turnover_cap
            action = self.position + trade

        return action

    def _compute_reward(self, pnl, cost, trade_size):
        """
        Compute reward with penalties.
        reward = pnl - cost - risk_penalty * var - dd_penalty * dd - turnover_penalty * turnover
        """
        risk_penalty = self.config['reward']['risk_penalty']
        dd_penalty = self.config['reward']['drawdown_penalty']
        turnover_penalty = self.config['reward']['turnover_penalty']

        # Risk penalty (approximate with recent vol)
        risk_term = risk_penalty * self._compute_rolling_vol(self.current_idx)**2

        # Drawdown penalty
        dd = max(0, self.peak_equity - self.equity) / self.peak_equity
        dd_term = dd_penalty * dd

        # Turnover penalty
        turnover_term = turnover_penalty * trade_size

        reward = pnl - cost - risk_term - dd_term - turnover_term

        return reward

# CAUSALITY ASSERTIONS
def test_causality(env):
    """Test that environment respects causality."""
    print("[TEST] Running causality checks...")

    # Reset environment
    state = env.reset(100, 200)

    # Check that state only uses data up to current_idx
    t = env.current_idx

    # Feature extraction should not access future data
    # This is enforced by implementation, but we verify:
    assert t == 100, "Environment should start at start_idx"

    # Step forward and check timing
    action = 0.5
    next_state, reward, done, info = env.step(action)

    # After step, we should be at t+1
    assert env.current_idx == 101, "Environment should advance by 1 step"

    # Reward should use return at t+1 (which is return[101])
    # We can't check exact value, but we check that it's computed
    assert 'pnl' in info, "Info should contain pnl"

    print("[TEST] Causality checks passed.")

# Create environment and run tests
env = TradingEnvironment(market_data, CONFIG)
test_causality(env)

# Save environment spec
env_spec = {
    'version': 'v1.0',
    'state_variables': [
        'lagged_returns (20 lags)',
        'rolling_volatility (20-period)',
        'regime_probabilities (2 regimes, filtered)',
        'current_position',
        'equity',
        'drawdown',
        'cumulative_turnover'
    ],
    'state_dimension': env._get_state().shape[0],
    'action_space': {
        'type': 'continuous',
        'bounds': CONFIG['constraints']['position_bounds']
    },
    'reward_formula': 'pnl - cost - risk_penalty * vol^2 - dd_penalty * drawdown - turnover_penalty * turnover',
    'constraints': CONFIG['constraints'],
    'timing': {
        'decision': 't',
        'execution': 't+1',
        'reward_realization': 't+1'
    },
    'causality_guarantee': 'All features use data <= t only. Filtered regime estimates only.'
}

env_spec_path = f"{PATHS['artifacts']}/environment_spec.json"
with open(env_spec_path, 'w') as f:
    json.dump(env_spec, f, indent=2)

print(f"[ENV] Environment spec saved. State dim: {env_spec['state_dimension']}")


[TEST] Running causality checks...
[TEST] Causality checks passed.
[ENV] Environment spec saved. State dim: 27


##7.BASELINES

###7.1.OVERVIEW


Before training any RL agent, we need to establish performance benchmarks. This section implements
five baseline strategies that represent different trading philosophies, from passive to rule-based
to greedy optimization. These baselines serve three critical purposes: they provide context for
evaluating the RL agent, they generate expert demonstrations for behavior cloning, and they reveal
what's achievable without sophisticated learning algorithms.

**Why Baselines Are Non-Negotiable**

A common mistake in RL research is reporting that "our agent achieved 15% returns" without context.
Is 15% good? It depends—what did simple alternatives achieve? If buy-and-hold earned 20% with lower
risk, your fancy RL agent is worthless. Baselines transform absolute performance metrics into
relative assessments: the RL agent must beat sensible alternatives to justify its complexity.

Moreover, baselines expose environment bugs. If even the simplest strategy produces nonsensical
results, something is wrong with the environment, costs, or reward function. It's much easier to
debug a 3-line baseline than a complex RL algorithm.

**Baseline 1: Cash (Do-Nothing)**

The simplest possible strategy: hold zero position at all times. This earns exactly 0% return and
incurs zero costs. It represents the null hypothesis—the performance floor that any active strategy
must beat. Surprisingly, many trading strategies fail to beat cash after accounting for costs and
risk.

Cash also serves as the reference point for Sharpe ratio calculations. If your strategy earns 5%
but with 20% volatility (Sharpe = 0.25), you might be better off staying in cash and sleeping well.

**Baseline 2: Buy-and-Hold**

Take a constant long position (+1.0) and hold forever. This captures pure market beta—you earn
whatever the market delivers, minus the initial transaction cost to establish the position. In
trending markets, buy-and-hold can be surprisingly effective. In mean-reverting or declining markets,
it suffers.

This baseline is particularly important for our synthetic data because we've embedded small positive
drift in both regimes. Buy-and-hold should earn positive returns on average, though it will experience
drawdowns during high-volatility periods. If buy-and-hold produces negative returns, we've
misconfigured our market generator.

**Baseline 3: Trend Following with Volatility Targeting**

This rule-based strategy represents classic technical analysis: compute the average of recent returns
(lookback window), take positions in the direction of this trend, and scale position size inversely
with volatility. When recent returns are positive, go long; when negative, go short. When volatility
is high, reduce position size; when low, increase it.

The volatility targeting is crucial—it implements rudimentary risk management without complex
optimization. By scaling exposure inversely with volatility, the strategy naturally reduces risk
during turbulent periods (high-vol regime) and increases it during calm periods (low-vol regime).

This baseline often performs remarkably well, especially in markets with momentum and regime
persistence. It will serve as our "expert" policy for behavior cloning because it encodes sensible
trading logic: follow trends, manage risk dynamically, respect volatility.

**Baseline 4: Myopic (Greedy One-Step)**

This strategy makes locally optimal decisions without considering long-term consequences. At each
timestep, it forecasts the next return (using average of recent returns), computes the position that
would maximize one-step-ahead expected reward (accounting for costs), and takes that position.

Myopic strategies are "greedy"—they optimize immediate payoff rather than long-run value. In some
environments this works well; in others it fails because it ignores delayed costs like drawdown
accumulation or constraint violations that hurt future options. This baseline tests whether
short-term optimization suffices or whether we need genuine sequential decision-making.

The implementation scales position by forecast strength and inverse volatility, then clips to
constraint bounds. It's essentially a simplified optimal execution strategy for a one-period horizon.

**Baseline 5: Imitation (Behavior Cloning Preview)**

This baseline will be added after we train the behavior cloning policy in Cell 8. It represents pure
imitation learning: the agent mimics the trend-following expert without attempting improvement. This
lets us measure the cost of imperfect imitation—how much performance degrades when we approximate
the expert with a parameterized policy rather than executing its logic directly.

**Evaluation Protocol and Metrics**

Each baseline runs on a common evaluation window (steps 100-1100), using the same environment, costs,
and constraints. We compute a comprehensive metric suite for each:

- **Net Return**: Final equity minus initial equity (1.0)
- **Volatility**: Standard deviation of equity returns
- **Sharpe Ratio**: Mean return divided by volatility (simple annualized form)
- **Maximum Drawdown**: Largest peak-to-trough decline in equity
- **Total Turnover**: Sum of absolute position changes
- **Average Cost**: Mean transaction cost per step
- **Constraint Violation Rate**: Fraction of steps where constraints were breached

These metrics capture both performance (returns, Sharpe) and implementation reality (turnover, costs,
violations). A strategy with high returns but catastrophic drawdowns or excessive violations is not
deployable, regardless of its Sharpe ratio.

**Visualization and Sanity Checks**

The section plots equity curves for all baselines on a single chart. This visual comparison is
incredibly informative:

- Do equity curves grow over time? (If not, our market has no edge to exploit)
- Which strategy handles the high-volatility regime better?
- Does trend-following outperform buy-and-hold? (It should, given our regime structure)
- Are there periods where all strategies fail simultaneously? (Suggests fundamental market difficulty)

These plots also reveal bugs immediately. If a baseline produces negative infinity equity or jumps
discontinuously, we have an implementation error.

**The Baseline Results as Ground Truth**

The JSON file saved at the end contains baseline metrics that become our performance targets. The RL
agent must beat trend-following (our best baseline) to justify its existence. If RL achieves 1.2x
return vs. trend's 1.39x, we've failed—the added complexity isn't worth the performance loss.

These baselines also calibrate our expectations. If the best baseline achieves Sharpe 90 (due to our
benign synthetic environment), we know this is an "easy" problem. On real data with Sharpe < 2, we'd
adjust our success criteria accordingly.

###7.2.CODE AND IMPLEMENTATION

In [6]:
# Cell 7 — Baseline strategies
def run_episode(env, policy_fn, start_idx, end_idx):
    """Run a single episode with given policy."""
    state = env.reset(start_idx, end_idx)
    done = False

    trajectory = {
        'states': [],
        'actions': [],
        'rewards': [],
        'infos': []
    }

    while not done:
        action = policy_fn(state, env)
        trajectory['states'].append(state)
        trajectory['actions'].append(action)

        next_state, reward, done, info = env.step(action)
        trajectory['rewards'].append(reward)
        trajectory['infos'].append(info)

        state = next_state

    return trajectory

def cash_baseline(state, env):
    """Do-nothing baseline: stay in cash."""
    return 0.0

def buy_hold_baseline(state, env):
    """Buy-and-hold baseline: constant position."""
    return env.pos_max

def trend_baseline(state, env):
    """Rule-based trend following with volatility targeting."""
    # Use mean of recent returns (from state's lagged returns)
    lagged_returns = state[:env.lookback]
    mean_return = np.mean(lagged_returns)

    # Volatility from state
    vol = state[env.lookback]  # rolling_vol is at this index

    # Vol targeting: scale position by inverse of vol
    target_vol = 0.02
    if vol > 0:
        scale = target_vol / vol
    else:
        scale = 1.0

    # Position = sign(trend) * scale
    if mean_return > 0:
        position = min(scale * env.pos_max, env.pos_max)
    elif mean_return < 0:
        position = max(-scale * env.pos_max, env.pos_min)
    else:
        position = 0.0

    return position

def myopic_baseline(state, env):
    """
    Myopic baseline: greedy one-step optimization.
    Use simple linear forecast of next return from state.
    """
    # Simple forecast: use mean of recent returns
    lagged_returns = state[:env.lookback]
    forecast = np.mean(lagged_returns)

    # Greedy position to maximize expected reward (with cost consideration)
    # If forecast > 0, go long; if < 0, go short
    # Scale by confidence (inverse of vol)
    vol = state[env.lookback]
    if vol > 0:
        scale = 1.0 / vol
    else:
        scale = 1.0

    position = np.clip(forecast * scale * 10, env.pos_min, env.pos_max)

    return position

def compute_metrics(trajectory):
    """
    Compute metrics from trajectory.
    CORRECTED: Proper Sharpe ratio calculation using mean and std of returns.
    """
    rewards = np.array(trajectory['rewards'])
    infos = trajectory['infos']

    # Extract equity curve
    equity = [info['equity'] for info in infos]
    equity = np.array(equity)

    # Net return
    net_return = equity[-1] - 1.0 if len(equity) > 0 else 0.0

    # Volatility and mean of returns
    if len(equity) > 1:
        equity_returns = np.diff(equity) / (equity[:-1] + 1e-6)
        mean_return = np.mean(equity_returns)
        vol = np.std(equity_returns)
    else:
        mean_return = 0.0
        vol = 0.0

    # Sharpe ratio (CORRECTED: mean_return / vol, not net_return / vol)
    sharpe = (mean_return / vol) if vol > 0 else 0.0

    # Max drawdown
    peak = np.maximum.accumulate(equity)
    drawdown = (peak - equity) / (peak + 1e-6)
    max_dd = np.max(drawdown) if len(drawdown) > 0 else 0.0

    # Turnover
    turnover = sum(info['trade_size'] for info in infos)

    # Average cost
    avg_cost = np.mean([info['cost'] for info in infos])

    # Constraint violations
    violations = sum(1 for info in infos if info['constraint_violation'])
    violation_rate = violations / len(infos) if len(infos) > 0 else 0.0

    return {
        'net_return': net_return,
        'volatility': vol,
        'sharpe': sharpe,
        'max_drawdown': max_dd,
        'turnover': turnover,
        'avg_cost': avg_cost,
        'violation_rate': violation_rate,
        'total_steps': len(infos)
    }

# Run baselines on full dataset
print("[BASELINES] Running baseline strategies...")

baseline_policies = {
    'cash': cash_baseline,
    'buy_hold': buy_hold_baseline,
    'trend': trend_baseline,
    'myopic': myopic_baseline
}

baseline_results = {}

# Use first 1000 steps for baseline evaluation
eval_start = 100
eval_end = 1100

for name, policy_fn in baseline_policies.items():
    print(f"[BASELINES] Running {name}...")
    trajectory = run_episode(env, policy_fn, eval_start, eval_end)
    metrics = compute_metrics(trajectory)
    baseline_results[name] = metrics
    print(f"  Net return: {metrics['net_return']:.4f}, Sharpe: {metrics['sharpe']:.4f}, Max DD: {metrics['max_drawdown']:.4f}")

# Save baseline metrics
baseline_path = f"{PATHS['artifacts']}/baseline_metrics.json"
with open(baseline_path, 'w') as f:
    json.dump(baseline_results, f, indent=2)

# Plot baseline equity curves
plt.figure(figsize=(12, 6))
for name, policy_fn in baseline_policies.items():
    trajectory = run_episode(env, policy_fn, eval_start, eval_end)
    equity = [info['equity'] for info in trajectory['infos']]
    plt.plot(equity, label=name)

plt.title('Baseline Strategy Equity Curves')
plt.xlabel('Time')
plt.ylabel('Equity')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig(f"{PATHS['plots']}/baseline_equity.png", dpi=100)
plt.close()

print("[BASELINES] Baseline results saved.")

[BASELINES] Running baseline strategies...
[BASELINES] Running cash...
  Net return: 0.0000, Sharpe: 0.0000, Max DD: 0.0000
[BASELINES] Running buy_hold...
  Net return: -0.0465, Sharpe: 0.0063, Max DD: 0.6621
[BASELINES] Running trend...
  Net return: 1.3890, Sharpe: 0.0648, Max DD: 0.4854
[BASELINES] Running myopic...
  Net return: 1.1644, Sharpe: 0.0565, Max DD: 0.4834
[BASELINES] Baseline results saved.


##8.OFF LINE  RL TRAINING

###8.1.OVERVIEW



This section implements the core RL training methodology: a two-stage offline learning approach that
combines behavior cloning with conservative policy improvement. Unlike online RL where agents explore
freely in live environments, offline RL must learn exclusively from historical data—a constraint that
matches real-world trading where experimentation with actual capital is prohibitively expensive and
risky.

**The Offline RL Challenge**

Imagine learning to drive by watching videos of expert drivers, without ever touching a steering wheel
until your first highway commute. That's offline RL. You can't experiment, can't try risky maneuvers
to see what happens, can't explore actions the expert never took. You must learn a good policy from
someone else's demonstrated behavior.

This creates a fundamental problem: if you try to learn a policy that's too different from the
demonstrations, you're optimizing in regions of state-action space where you have no data. Your
value estimates become wildly optimistic (the "extrapolation error" problem), and your learned policy
fails catastrophically when deployed. Conservative offline RL addresses this by staying close to the
demonstrated behavior while making small, justified improvements.

**Stage 1: Behavior Cloning (BC) - Learning from the Expert**

Behavior cloning is supervised learning applied to RL: we treat the expert's state-action pairs as
a dataset and train a policy to mimic them. Our expert is the trend-following baseline from Cell 7—
a sensible strategy that respects constraints and earns positive returns.

We collect expert trajectories by running the trend baseline for 8 episodes (each spanning 100 steps)
across the training window. This generates roughly 800 state-action pairs: states the expert
encountered and actions it chose. Our goal is to learn a parameterized policy (a linear model with
weights W and bias b) that approximates this expert behavior.

The training process minimizes mean squared error between the expert's actions and the policy's
predictions. Over 100 epochs, we perform stochastic gradient descent:

- Sample a random batch of state-action pairs
- Predict actions using current policy parameters
- Compute MSE loss between predictions and expert actions
- Calculate gradients of loss with respect to policy parameters
- Update parameters to reduce loss

The loss curve tracks training progress—it should decrease and stabilize. If loss remains high, our
linear model is too simple to capture the expert's logic. If loss crashes to zero too quickly, we're
overfitting.

**Why Linear Policies?**

We deliberately use a simple linear policy (action = W^T · state + b, clipped to bounds) rather than
deep neural networks. This transparency is pedagogical—you can inspect the learned weights and
understand what features the policy relies on. It's also practical—linear policies generalize better
from limited data and are less prone to the extrapolation errors that plague deep RL.

In production systems, simplicity is a feature, not a bug. A linear policy can be audited, stress-
tested, and debugged much more easily than a black-box neural network with millions of parameters.

**Stage 2: Conservative Policy Improvement (CPI) - Careful Optimization**

Behavior cloning gives us a safe baseline policy that mimics the expert. But we want more—can we
improve beyond the expert while staying safe? Conservative policy improvement achieves this through
constrained optimization: maximize expected reward while penalizing deviation from the BC policy.

The CPI algorithm performs a small number of gradient steps (5 in our configuration) where each step:

- Evaluates the current policy by running it in the environment
- Estimates the policy gradient (how to adjust parameters to increase reward)
- Computes a deviation penalty gradient (how much we're drifting from BC policy)
- Updates parameters in the direction that increases reward minus deviation penalty

The deviation penalty is crucial—it's a "trust region" that prevents the policy from wandering into
uncharted territory. We're essentially saying: "improve, but don't change so much that we're no
longer confident in our value estimates."

**Gradient Estimation via Finite Differences**

Since we're implementing RL from scratch without automatic differentiation libraries, we estimate
gradients using finite differences. For each policy parameter, we:

- Perturb it slightly upward (+epsilon)
- Measure the resulting average reward
- Perturb it downward (-epsilon)
- Measure the reward again
- Gradient ≈ (reward_up - reward_down) / (2 × epsilon)

This is computationally expensive (requires running the environment multiple times per parameter), so
we sample only a subset of parameters per iteration. In production systems, you'd use proper
automatic differentiation, but finite differences makes the learning process completely transparent.

**The Conservative Philosophy**

Notice we take only 5 CPI steps, not 100 or 1000. This conservatism is intentional. Each step moves
us farther from the demonstrated behavior, increasing extrapolation risk. After a few steps, we're
making decisions in states the expert rarely visited, with actions the expert rarely chose. Our
value estimates become unreliable, and further optimization is likely to find spurious patterns rather
than genuine improvements.

This mirrors how expert traders develop new strategies: start with proven techniques, make small
modifications, validate extensively, and stop before you've "optimized yourself into a corner" by
overfitting to historical quirks.

**Training Traces and Transparency**

Throughout training, we log everything: BC loss per epoch, CPI reward per step, number of training
samples, episodes collected. These traces get saved to JSON and plotted for visual inspection. The
plots reveal:

- Does BC loss converge smoothly? (Indicates successful imitation learning)
- Does CPI improve reward over BC? (Validates the improvement mechanism)
- Are improvements monotonic or noisy? (Suggests stability or instability)

If CPI reward decreases or fluctuates wildly, something is wrong—bad gradient estimates, poor learning
rate, or the policy is leaving the safe region.

**Policy Serialization and Governance**

Both the BC policy and CPI policy get saved to disk as .npz files containing the learned weights and
bias. This creates reproducible policy artifacts—we can reload these exact policies months later and
get identical behavior. Each policy file is time-stamped and linked to the run manifest, creating a
complete audit trail from training data to learned parameters to evaluation results.

**The Pedagogical Payoff**

By implementing RL transparently rather than calling library functions, we've demystified the learning
process. Behavior cloning is just supervised learning. Policy improvement is just gradient ascent with
a penalty term. There's no magic—just optimization under constraints, implemented with basic NumPy
operations that you can inspect, modify, and trust.

###8.2.CODE AND IMPLEMENTATION

In [7]:

class LinearPolicy:
    """
    Simple linear policy for continuous actions.
    action = clip(W^T state + b, pos_min, pos_max)
    """

    def __init__(self, state_dim, pos_min, pos_max, seed=None):
        if seed is not None:
            np.random.seed(seed)
        self.W = np.random.randn(state_dim) * 0.01
        self.b = 0.0
        self.pos_min = pos_min
        self.pos_max = pos_max

    def predict(self, state):
        """Predict action for given state."""
        action = np.dot(self.W, state) + self.b
        return np.clip(action, self.pos_min, self.pos_max)

    def get_params(self):
        """Get policy parameters."""
        return {'W': self.W.copy(), 'b': self.b}

    def set_params(self, params):
        """Set policy parameters."""
        self.W = params['W'].copy()
        self.b = params['b']

def collect_expert_trajectories(env, expert_policy_fn, n_episodes, start_idx, end_idx, step=100):
    """Collect trajectories from expert policy."""
    trajectories = []

    for i in range(n_episodes):
        ep_start = start_idx + i * step
        ep_end = min(ep_start + step, end_idx)
        if ep_end - ep_start < 50:
            break

        traj = run_episode(env, expert_policy_fn, ep_start, ep_end)
        trajectories.append(traj)

    return trajectories

def train_behavior_cloning(env, expert_trajectories, config):
    """
    Train policy via behavior cloning.
    Minimize MSE between policy actions and expert actions.
    """
    print("[BC] Training behavior cloning policy...")

    # Collect all state-action pairs
    states = []
    actions = []
    for traj in expert_trajectories:
        states.extend(traj['states'])
        actions.extend(traj['actions'])

    states = np.array(states)
    actions = np.array(actions)
    n_samples = len(states)

    print(f"[BC] Training on {n_samples} samples")

    # Initialize policy
    state_dim = states.shape[1]
    policy = LinearPolicy(state_dim, env.pos_min, env.pos_max, seed=config['training']['seed_train'])

    # Training hyperparameters
    epochs = config['training']['bc_epochs']
    lr = config['training']['bc_lr']
    batch_size = config['training']['bc_batch_size']

    losses = []

    for epoch in range(epochs):
        # Shuffle data
        indices = np.random.permutation(n_samples)
        epoch_loss = 0.0
        n_batches = 0

        for i in range(0, n_samples, batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_states = states[batch_indices]
            batch_actions = actions[batch_indices]

            # Forward pass
            predictions = np.array([policy.predict(s) for s in batch_states])

            # MSE loss
            loss = np.mean((predictions - batch_actions)**2)
            epoch_loss += loss
            n_batches += 1

            # Gradient (for linear policy)
            # dL/dW = 2/n * sum((pred - target) * state)
            errors = predictions - batch_actions
            grad_W = (2.0 / len(batch_states)) * np.sum([errors[j] * batch_states[j] for j in range(len(errors))], axis=0)
            grad_b = (2.0 / len(batch_states)) * np.sum(errors)

            # Update
            policy.W -= lr * grad_W
            policy.b -= lr * grad_b

        avg_loss = epoch_loss / n_batches
        losses.append(avg_loss)

        if (epoch + 1) % 20 == 0:
            print(f"  Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")

    print("[BC] Behavior cloning complete.")

    return policy, losses

def conservative_policy_improvement(env, bc_policy, train_start, train_end, config):
    """
    Conservative policy improvement: small gradient steps that stay close to BC policy.
    Maximize reward while penalizing deviation from BC.

    Fixed to properly use integer indices.
    """
    print("[CPI] Starting conservative policy improvement...")

    # Clone BC policy
    cpi_policy = LinearPolicy(bc_policy.W.shape[0], env.pos_min, env.pos_max)
    cpi_policy.set_params(bc_policy.get_params())

    # CPI hyperparameters
    cpi_steps = config['training']['cpi_steps']
    cpi_lr = config['training']['cpi_lr']
    deviation_penalty = config['training']['cpi_deviation_penalty']

    rewards_history = []

    # Use proper integer indices for evaluation
    eval_start = int(train_start)
    eval_end = int(train_end)

    for step in range(cpi_steps):
        # Evaluate current policy
        traj = run_episode(env, lambda s, e: cpi_policy.predict(s), eval_start, eval_end)
        avg_reward = np.mean(traj['rewards'])
        rewards_history.append(avg_reward)

        # Estimate gradient via finite differences (sample subset for efficiency)
        epsilon = 0.01
        grad_W = np.zeros_like(cpi_policy.W)

        # Sample only a few dimensions to estimate gradient
        n_dims_sample = min(5, len(cpi_policy.W))
        sampled_dims = np.random.choice(len(cpi_policy.W), n_dims_sample, replace=False)

        for i in sampled_dims:
            # Perturb parameter i
            cpi_policy.W[i] += epsilon
            traj_plus = run_episode(env, lambda s, e: cpi_policy.predict(s), eval_start, eval_end)
            reward_plus = np.mean(traj_plus['rewards'])

            cpi_policy.W[i] -= 2 * epsilon
            traj_minus = run_episode(env, lambda s, e: cpi_policy.predict(s), eval_start, eval_end)
            reward_minus = np.mean(traj_minus['rewards'])

            # Restore
            cpi_policy.W[i] += epsilon

            # Gradient
            grad_W[i] = (reward_plus - reward_minus) / (2 * epsilon)

        # Deviation penalty gradient
        deviation = cpi_policy.W - bc_policy.W
        grad_deviation = 2 * deviation_penalty * deviation

        # Update
        cpi_policy.W += cpi_lr * (grad_W - grad_deviation)

        print(f"  CPI step {step+1}/{cpi_steps}, Avg reward: {avg_reward:.6f}")

    print("[CPI] Conservative policy improvement complete.")

    return cpi_policy, rewards_history

# Training window
train_start = 100
train_end = 900

# Collect expert trajectories (use trend baseline as expert)
print("[TRAIN] Collecting expert trajectories...")
expert_trajectories = collect_expert_trajectories(
    env, trend_baseline, n_episodes=8, start_idx=train_start, end_idx=train_end, step=100
)
print(f"[TRAIN] Collected {len(expert_trajectories)} expert trajectories")

# Train BC policy
bc_policy, bc_losses = train_behavior_cloning(env, expert_trajectories, CONFIG)

# Save BC policy
bc_policy_path = f"{PATHS['policy']}/bc_policy.npz"
np.savez(bc_policy_path, W=bc_policy.W, b=np.array([bc_policy.b]))
print(f"[TRAIN] BC policy saved to {bc_policy_path}")

# Train CPI policy (FIXED: pass integer indices directly)
cpi_policy, cpi_rewards = conservative_policy_improvement(env, bc_policy, train_start, train_end, CONFIG)

# Save CPI policy
cpi_policy_path = f"{PATHS['policy']}/cpi_policy.npz"
np.savez(cpi_policy_path, W=cpi_policy.W, b=np.array([cpi_policy.b]))
print(f"[TRAIN] CPI policy saved to {cpi_policy_path}")

# Save training traces
training_traces = {
    'bc_losses': [float(x) for x in bc_losses],
    'cpi_rewards': [float(x) for x in cpi_rewards],
    'n_expert_trajectories': len(expert_trajectories),
    'total_expert_samples': sum(len(t['states']) for t in expert_trajectories)
}

traces_path = f"{PATHS['logs']}/training_traces.json"
with open(traces_path, 'w') as f:
    json.dump(training_traces, f, indent=2)

# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(bc_losses)
axes[0].set_title('Behavior Cloning Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(cpi_rewards)
axes[1].set_title('CPI Average Reward')
axes[1].set_xlabel('CPI Step')
axes[1].set_ylabel('Avg Reward')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{PATHS['plots']}/training_curves.png", dpi=100)
plt.close()

print("[TRAIN] Training complete. Traces saved.")

# Add imitation baseline to baseline_policies
baseline_policies['imitation'] = lambda s, e: bc_policy.predict(s)


[TRAIN] Collecting expert trajectories...
[TRAIN] Collected 8 expert trajectories
[BC] Training behavior cloning policy...
[BC] Training on 792 samples
  Epoch 20/100, Loss: 0.368709
  Epoch 40/100, Loss: 0.372289
  Epoch 60/100, Loss: 0.370607
  Epoch 80/100, Loss: 0.366539
  Epoch 100/100, Loss: 0.379618
[BC] Behavior cloning complete.
[TRAIN] BC policy saved to /content/ch19_runs/20251229_173257/policy/bc_policy.npz
[CPI] Starting conservative policy improvement...
  CPI step 1/5, Avg reward: -0.196414
  CPI step 2/5, Avg reward: -0.196412
  CPI step 3/5, Avg reward: -0.183177
  CPI step 4/5, Avg reward: -0.173226
  CPI step 5/5, Avg reward: -0.173443
[CPI] Conservative policy improvement complete.
[TRAIN] CPI policy saved to /content/ch19_runs/20251229_173257/policy/cpi_policy.npz
[TRAIN] Training complete. Traces saved.


##9.WALK FORWARD EVALUATION

###9.1.OVERVIEW


This section implements the gold standard for trading strategy validation: walk-forward analysis.
Unlike simple train-test splits that evaluate on a single held-out period, walk-forward testing
simulates the realistic scenario where you repeatedly retrain your model on growing historical data
and evaluate it on the immediate future. This catches overfitting, reveals performance degradation
over time, and tests whether your strategy adapts to changing market conditions.

**The Walk-Forward Methodology**

Think of walk-forward testing as a rolling window that moves through time:

- **Fold 0**: Train on steps 100-900, test on steps 900-1100
- **Fold 1**: Train on steps 300-1100, test on steps 1100-1300  
- **Fold 2**: Train on steps 500-1300, test on steps 1300-1500

Each fold shifts forward by the step length (200 steps in our configuration), creating overlapping
training windows but non-overlapping test windows. This mimics how you'd deploy a strategy in
production: train on all available history, trade live for a period, then retrain with the new data
included.

The key insight is that test periods are strictly out-of-sample—they occur after training data ends.
The policy makes predictions about periods it has never seen, using a model trained before those
periods existed. This eliminates look-ahead bias completely.

**Why Single Train-Test Splits Fail**

A single split is dangerous because it might get lucky. Perhaps your test period happened to be
unusually favorable for your strategy. Or perhaps your model picked up patterns specific to your
particular split point. Walk-forward testing averages over multiple test periods, revealing whether
performance is robust or fragile.

Moreover, markets are non-stationary—patterns that worked in 2020 may fail in 2023. Walk-forward
testing forces your strategy to prove itself across different regimes represented in different folds.
If performance collapses in fold 2, you know your strategy doesn't generalize across time.

**The Evaluation Protocol**

For each fold, we evaluate six policies: cash, buy-and-hold, trend, myopic, BC policy, and CPI
policy. All policies use the same environment, constraints, costs, and test window—the only difference
is the decision rule. This creates an apples-to-apples comparison.

For our RL policies (BC and CPI), note that we don't retrain them per fold in this implementation—
we trained them once in Cell 8 and now evaluate the frozen policies across all folds. In a production
system, you'd retrain per fold using only data available up to that fold's training cutoff. Our
simplified approach still captures the essential insight: does the policy work out-of-sample?

**Comprehensive Metrics Per Fold**

For each policy on each fold, we compute the full metric suite:

- **Net Return**: Absolute performance measure—did you make money?
- **Sharpe Ratio**: Risk-adjusted performance—did you make money efficiently?
- **Maximum Drawdown**: Worst-case risk measure—how much could you lose from peak?
- **Turnover**: Trading intensity—how much churn?
- **Average Cost**: Transaction cost burden—how much went to friction?
- **Violation Rate**: Constraint compliance—did you breach risk limits?

These metrics tell different stories. A policy might have high returns but terrible drawdowns (not
deployable). Another might have mediocre returns but excellent Sharpe and low turnover (highly
deployable). The multi-metric view prevents optimizing for a single number while ignoring practical
constraints.

**Cross-Fold Performance Patterns**

The evaluation results saved to JSON allow us to analyze patterns across folds:

- **Consistency**: Does the RL policy beat baselines in all folds or just some? Consistent
outperformance suggests robust advantage; inconsistent results suggest luck or overfitting.

- **Degradation**: Does performance decline from early folds to late folds? This signals that the
policy is becoming obsolete as market conditions drift away from training data characteristics.

- **Relative Rankings**: Do the same strategies dominate across folds? If trend beats myopic in
fold 0 but loses in fold 2, the market structure has fundamentally changed.

- **Drawdown Timing**: Do all strategies suffer drawdowns simultaneously? This indicates systematic
market difficulty (unavoidable). If only your RL policy suffers while baselines survive, your
policy has a specific vulnerability.

**Equity Curve Visualization**

The per-fold equity curve plots provide intuitive visual assessment. We plot trend, BC policy, and
CPI policy on each fold:

- Smooth upward slopes indicate consistent profitability
- Flat regions indicate periods of poor performance or churning
- Sharp drops indicate drawdown events
- Crossing curves show relative performance shifts

If the CPI policy equity curve dominates trend throughout, we have visual evidence of improvement.
If curves cross repeatedly, performance is regime-dependent and neither strategy is uniformly superior.

**The Three-Fold Limit**

We limit evaluation to three folds for computational efficiency in this pedagogical notebook. In
production, you'd run dozens of folds covering years of history. More folds provide better statistical
power to distinguish true alpha from noise. Three folds suffices to demonstrate the methodology and
catch egregious failures.

**What Success Looks Like**

A successful RL policy should:

- Beat the cash baseline in all folds (otherwise, why trade?)
- Beat or match the trend baseline in most folds (otherwise, why bother with RL?)
- Maintain acceptable drawdowns (< 50% in our environment)
- Keep turnover reasonable (not churning for no reason)
- Show stable performance across folds (not getting lucky once)

If the CPI policy beats trend by 10% in fold 0 but loses by 20% in fold 2, we don't have a
deployable strategy—we have an unstable optimization that got lucky on specific data.

**The Uncomfortable Truth**

Walk-forward testing often delivers bad news. Strategies that looked amazing on a single backtest
reveal themselves as fragile when tested across multiple out-of-sample periods. This is a feature,
not a bug. Better to discover fragility in simulation than in production with real capital. The
walk-forward methodology is your last line of defense against delusional overconfidence from
in-sample optimization.

###9.2.CODE AND IMPLEMENTATION

In [8]:
# Cell 9 — Walk-forward evaluation
def walk_forward_evaluation(env, policies, config):
    """
    Perform walk-forward evaluation.
    For each fold: train on window, evaluate on forward test window.
    """
    print("[EVAL] Starting walk-forward evaluation...")

    eval_config = config['evaluation']
    train_len = eval_config['train_len']
    test_len = eval_config['test_len']
    step_len = eval_config['step_len']
    min_start = eval_config['min_train_start']

    T = env.T
    results = []

    fold = 0
    train_start = min_start

    while train_start + train_len + test_len < T:
        train_end = train_start + train_len
        test_start = train_end
        test_end = test_start + test_len

        print(f"[EVAL] Fold {fold}: train [{train_start}, {train_end}], test [{test_start}, {test_end}]")

        fold_results = {
            'fold': fold,
            'train_start': train_start,
            'train_end': train_end,
            'test_start': test_start,
            'test_end': test_end,
            'policies': {}
        }

        # Evaluate each policy on test window
        for name, policy_fn in policies.items():
            traj = run_episode(env, policy_fn, test_start, test_end)
            metrics = compute_metrics(traj)
            fold_results['policies'][name] = metrics
            print(f"  {name}: return={metrics['net_return']:.4f}, sharpe={metrics['sharpe']:.4f}")

        results.append(fold_results)

        # Move to next fold
        train_start += step_len
        fold += 1

        if fold >= 3:  # Limit to 3 folds for demo
            break

    return results

# Create policy dictionary for evaluation
eval_policies = {
    'cash': cash_baseline,
    'buy_hold': buy_hold_baseline,
    'trend': trend_baseline,
    'myopic': myopic_baseline,
    'bc_policy': lambda s, e: bc_policy.predict(s),
    'cpi_policy': lambda s, e: cpi_policy.predict(s)
}

# Run walk-forward evaluation
wf_results = walk_forward_evaluation(env, eval_policies, CONFIG)

# Save evaluation results
eval_path = f"{PATHS['artifacts']}/evaluation_suite.json"
with open(eval_path, 'w') as f:
    json.dump(wf_results, f, indent=2, default=float)

# Plot per-fold equity curves for RL policies
n_folds = len(wf_results)
fig, axes = plt.subplots(n_folds, 1, figsize=(12, 4*n_folds))
if n_folds == 1:
    axes = [axes]

for i, fold_result in enumerate(wf_results):
    test_start = fold_result['test_start']
    test_end = fold_result['test_end']

    for name in ['trend', 'bc_policy', 'cpi_policy']:
        if name in eval_policies:
            traj = run_episode(env, eval_policies[name], test_start, test_end)
            equity = [info['equity'] for info in traj['infos']]
            axes[i].plot(equity, label=name)

    axes[i].set_title(f"Fold {i} Equity Curves")
    axes[i].set_ylabel('Equity')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

axes[-1].set_xlabel('Time')
plt.tight_layout()
plt.savefig(f"{PATHS['plots']}/walkforward_equity.png", dpi=100)
plt.close()

print("[EVAL] Walk-forward evaluation complete.")

[EVAL] Starting walk-forward evaluation...
[EVAL] Fold 0: train [100, 900], test [900, 1100]
  cash: return=0.0000, sharpe=0.0000
  buy_hold: return=-0.1650, sharpe=-0.0684
  trend: return=0.1988, sharpe=0.0859
  myopic: return=0.1295, sharpe=0.0606
  bc_policy: return=0.0335, sharpe=0.0430
  cpi_policy: return=0.0304, sharpe=0.0440
[EVAL] Fold 1: train [300, 1100], test [1100, 1300]
  cash: return=0.0000, sharpe=0.0000
  buy_hold: return=0.0932, sharpe=0.0361
  trend: return=-0.3037, sharpe=-0.1233
  myopic: return=-0.2913, sharpe=-0.1329
  bc_policy: return=-0.0311, sharpe=-0.0387
  cpi_policy: return=-0.0273, sharpe=-0.0388
[EVAL] Fold 2: train [500, 1300], test [1300, 1500]
  cash: return=0.0000, sharpe=0.0000
  buy_hold: return=-0.2314, sharpe=-0.0587
  trend: return=-0.1371, sharpe=-0.0457
  myopic: return=-0.2617, sharpe=-0.0921
  bc_policy: return=0.0355, sharpe=0.0384
  cpi_policy: return=0.0302, sharpe=0.0375
[EVAL] Walk-forward evaluation complete.


##10.OFF POLICY EVALUATION

###10.1.OVERVIEW

**Cell 10: Off-Policy Evaluation Pitfalls and the Importance Sampling Trap**

This section demonstrates one of the most dangerous misconceptions in offline RL: the belief that
you can reliably estimate how a new policy will perform by replaying historical data collected under
a different policy. This technique—off-policy evaluation (OPE)—is theoretically elegant but
practically treacherous. We show exactly why naive OPE fails and why walk-forward backtests remain
irreplaceable despite their limitations.

**The Off-Policy Evaluation Dream**

Imagine you have historical data from trading with strategy A (the "behavior policy"). You develop
a new strategy B (the "target policy"). Can you estimate B's performance without actually trading it?
If yes, you could test thousands of strategies instantly, finding winners without risking capital.
This is the OPE promise.

The standard approach is importance sampling: reweight the observed rewards by the probability ratio
of actions under the two policies. If an action was likely under B but unlikely under A, upweight
its reward. If unlikely under B but common under A, downweight it. In theory, this corrects for the
distribution mismatch and gives an unbiased estimate of B's expected return.

**Why It Fails: The Distribution Mismatch Problem**

The demonstration uses our trend-following baseline as the behavior policy and the CPI policy as
the target. We run the behavior policy through the environment, collecting a trajectory of states,
actions, and rewards. Then we compute what actions the target policy would have chosen in those
same states.

The first red flag appears in the "support" metric: how often does the target policy choose actions
similar to the behavior policy? We compute the average absolute difference between target and
behavior actions, and the fraction of timesteps where they differ by less than 0.1 units.

If support is low—meaning the policies frequently disagree—importance sampling becomes dangerous.
You're trying to estimate performance in regions of action space that the behavior policy rarely
explored. Your weights will be extreme, and your estimates will be unreliable.

**The Importance Sampling Weight Explosion**

For continuous action spaces, we approximate importance sampling using Gaussian likelihoods. The
weight for each timestep is:

**weight_t = P(observed_action | state, target_policy) / P(observed_action | state, behavior_policy)**

When the target policy strongly prefers a different action than what was observed, this weight
becomes very large. When it strongly dislikes the observed action, the weight becomes very small.
The product of these weights across timesteps compounds the problem—a few large weights can dominate
the entire estimate.

We compute the importance sampling estimate of the target policy's return by multiplying each
observed reward by its weight and summing. We also compute the variance of weights—a diagnostic for
estimator quality. High variance means the estimate is dominated by a few lucky (or unlucky)
timesteps rather than being a stable average.

**Ground Truth Comparison**

The critical step is computing the true target policy return by actually running it in the environment.
This gives us ground truth for comparison. The importance sampling error is the absolute difference
between the IS estimate and the true return.

In typical cases, the IS estimate is wildly off—sometimes predicting 50% higher returns than reality,
sometimes 30% lower. The estimator is not just noisy (which we could tolerate with enough data)—it's
biased because the policies operate in different regions of state-action space.

**The Weight Variance Plot**

The visualization of importance sampling weights over time is particularly revealing. Weights should
hover around 1.0 if the policies are similar. Instead, we often see:

- Weights ranging from 0.01 to 100 or more
- Sudden spikes where a single timestep gets enormous weight
- Long stretches where weights are near zero (the observed actions were "surprising" under the
target policy)

This instability means a few timesteps dominate the estimate. If those timesteps happened to have
unusually good or bad rewards by chance, the entire estimate is contaminated. Adding more data
doesn't help—the fundamental problem is distribution mismatch, not sample size.

**Why This Matters for Trading**

In academic RL benchmarks (robot control, games), policies often share similar action distributions
because they're solving the same task. In trading, different strategies can be radically different:

- A trend-follower takes large positions during momentum
- A mean-reversion trader does the opposite
- A risk-parity strategy rebalances to constant volatility

Trying to evaluate a mean-reversion policy using trend-following data is hopeless—the policies
disagree on almost every state. The importance weights explode, and estimates become meaningless.

**The Conservative Alternative: Bounded Estimates**

The demonstration saves an OPE demo report with a stark warning: "Naive OPE can be highly biased
and high-variance when policies differ significantly." We also compute the maximum importance weight
and weight variance as diagnostic metrics.

Some research proposes conservative OPE methods that provide lower bounds on performance rather than
point estimates. If you can prove the target policy will earn at least X%, that's actionable even if
you can't pin down the exact value. But even these methods struggle when distribution mismatch is
severe.

**The Uncomfortable Conclusion**

Off-policy evaluation is not a substitute for actual backtesting. You cannot reliably predict how a
new trading strategy will perform by analyzing old data collected under a different strategy. The
math looks elegant, but the assumptions (sufficient overlap between behavior and target distributions)
are routinely violated in practice.

This is why Cell 9's walk-forward backtests are essential. We actually run the policy in the
environment, observing what it does and what rewards it receives. There's no distribution mismatch,
no importance weights, no extrapolation. The backtest is a direct simulation of deployment.

Yes, backtests have their own problems—they assume our environment model (costs, execution, market
dynamics) is correct. But at least the errors are about modeling accuracy, not statistical estimation
failure. We'd rather have an accurate estimate of an approximate model than a wildly inaccurate
estimate of the true model.

**The Pedagogical Message**

This section teaches healthy skepticism. When someone claims their RL trading system will earn 30%
based on off-policy evaluation, demand to see walk-forward backtests. When a paper reports impressive
OPE results, check whether behavior and target policies are suspiciously similar. OPE is a useful
diagnostic tool for spotting obviously terrible policies, but it's not a substitute for rigorous
out-of-sample testing.

###10.2.CODE AND IMPLEMENTATION

In [9]:

# Cell 10 — OPE demonstration (why naive replay misleads)
def demonstrate_ope_pitfalls(env, behavior_policy, target_policy, start_idx, end_idx):
    """
    Demonstrate why naive off-policy evaluation can mislead.
    """
    print("[OPE] Demonstrating OPE pitfalls...")

    # Collect trajectory under behavior policy
    behavior_traj = run_episode(env, behavior_policy, start_idx, end_idx)
    behavior_return = sum(behavior_traj['rewards'])

    print(f"[OPE] Behavior policy return: {behavior_return:.4f}")

    # Naive replay: assume target policy would get same rewards
    # (This is wrong if policies differ significantly)
    target_actions = [target_policy(s, env) for s in behavior_traj['states']]
    behavior_actions = behavior_traj['actions']

    # Compute "support" metric: how often target chooses actions similar to behavior
    action_diffs = [abs(ta - ba) for ta, ba in zip(target_actions, behavior_actions)]
    avg_diff = np.mean(action_diffs)
    support_rate = np.mean([1 if d < 0.1 else 0 for d in action_diffs])

    print(f"[OPE] Target vs behavior action difference: {avg_diff:.4f}")
    print(f"[OPE] Support rate (actions within 0.1): {support_rate:.4f}")

    # Simple importance sampling (for short horizon)
    # Weight = P(action | state, target) / P(action | state, behavior)
    # For continuous actions, use Gaussian likelihood approximation

    # Compute IS weights (simplified: assume Gaussian with fixed std)
    std = 0.1
    is_weights = []
    for ta, ba in zip(target_actions, behavior_actions):
        # P(action) ~ exp(-0.5 * (action - policy_mean)^2 / std^2)
        log_target = -0.5 * ((ba - ta) / std)**2
        log_behavior = -0.5 * ((ba - ba) / std)**2  # Always 0
        weight = np.exp(log_target - log_behavior)
        is_weights.append(weight)

    is_weights = np.array(is_weights)

    # IS estimate of target return
    is_estimate = np.sum(is_weights * np.array(behavior_traj['rewards']))

    print(f"[OPE] IS estimate of target return: {is_estimate:.4f}")
    print(f"[OPE] IS weight variance: {np.var(is_weights):.4f}")

    # True target policy return (ground truth)
    target_traj = run_episode(env, target_policy, start_idx, end_idx)
    target_return = sum(target_traj['rewards'])

    print(f"[OPE] True target policy return: {target_return:.4f}")
    print(f"[OPE] IS error: {abs(is_estimate - target_return):.4f}")

    # Save OPE demo
    ope_demo = {
        'behavior_return': float(behavior_return),
        'target_return': float(target_return),
        'is_estimate': float(is_estimate),
        'is_error': float(abs(is_estimate - target_return)),
        'avg_action_diff': float(avg_diff),
        'support_rate': float(support_rate),
        'is_weight_variance': float(np.var(is_weights)),
        'is_weight_max': float(np.max(is_weights)),
        'warning': 'Naive OPE can be highly biased and high-variance when policies differ significantly.'
    }

    ope_path = f"{PATHS['artifacts']}/ope_demo.json"
    with open(ope_path, 'w') as f:
        json.dump(ope_demo, f, indent=2)

    # Plot IS weights
    plt.figure(figsize=(10, 4))
    plt.plot(is_weights)
    plt.axhline(y=1.0, color='r', linestyle='--', label='Weight=1')
    plt.title('Importance Sampling Weights Over Time')
    plt.xlabel('Time')
    plt.ylabel('IS Weight')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(f"{PATHS['plots']}/ope_weights.png", dpi=100)
    plt.close()

    print("[OPE] OPE demonstration complete.")

# Run OPE demo
demonstrate_ope_pitfalls(
    env,
    behavior_policy=lambda s, e: trend_baseline(s, e),
    target_policy=lambda s, e: cpi_policy.predict(s),
    start_idx=500,
    end_idx=600
)


[OPE] Demonstrating OPE pitfalls...
[OPE] Behavior policy return: -4.1723
[OPE] Target vs behavior action difference: 0.4944
[OPE] Support rate (actions within 0.1): 0.0909
[OPE] IS estimate of target return: -0.3595
[OPE] IS weight variance: 0.0550
[OPE] True target policy return: -3.4467
[OPE] IS error: 3.0872
[OPE] OPE demonstration complete.


##11.STRESS TEST AND SENSITIVITY

###11.1.OVERVIEW



This section moves beyond evaluating average-case performance to systematically stress-testing our
RL policies under adverse conditions. In production trading, strategies rarely fail during normal
markets—they fail when costs spike, liquidity evaporates, execution lags increase, or market regimes
shift. Stress testing reveals these vulnerabilities before they destroy real capital, transforming
abstract performance metrics into actionable risk assessments.

**The Stress Testing Philosophy**

Backtests tell you what happened in one particular history. Stress tests tell you what could happen
under conditions your training data didn't cover. This is crucial because:

- Markets are non-stationary—future conditions will differ from the past
- Tail events (crises, flash crashes) are underrepresented in historical data
- Operational reality often diverges from assumptions (costs increase, systems slow down)
- Regulatory changes or market structure shifts can invalidate assumptions overnight

A strategy that looks robust in backtests but collapses under 2× cost inflation is not deployable.
Stress testing exposes these fragilities while you can still fix them—or decide not to deploy.

**Stress Test 1: Cost Inflation**

We systematically inflate all cost components (fees, spreads, impact) by factors of 1.0×, 1.5×, and
2.0×. This simulates:

- Moving to a more expensive broker or exchange
- Trading during periods of elevated bid-ask spreads
- Scaling up position sizes where impact costs grow superlinearly
- Regulatory changes that impose higher transaction taxes

We re-run all policies under each cost scenario and compare performance degradation. A robust policy
should gracefully degrade—returns decline, but not catastrophically. A fragile policy might flip
from +20% to -10% returns under 2× costs, indicating it was profitable only because we underestimated
friction.

The key insight: high-turnover strategies are disproportionately sensitive to cost inflation. If your
RL policy trades daily and each round-trip costs 15 bps at 1× costs, that's 37.5% annual cost drag.
At 2× costs, you're burning 75% annually before earning any market returns. Meanwhile, a low-turnover
baseline might barely notice the cost increase.

**Stress Test 2: Latency Shifts**

Our baseline assumption is execution at t+1 (one-step delay). But what if your system slows down and
executes at t+2? Or network congestion adds variable delays? This stress test acknowledges that
execution lag is not a constant—it varies with market conditions, system load, and infrastructure
reliability.

We note this stress test as a "placeholder" in the implementation because fully implementing variable
latency requires modifying the environment step function. In production systems, you'd run Monte
Carlo simulations with latency drawn from empirical distributions measured in your actual trading
infrastructure.

The pedagogical point stands: latency kills alpha. If your strategy relies on rapidly exploiting
mean reversion, adding one extra step of delay might eliminate all profits. Signal-to-noise ratio
decays exponentially with latency in high-frequency contexts, and even daily strategies suffer if
orders sit in queues during volatile periods.

**Stress Test 3: Liquidity Shocks**

We multiply the liquidity proxy by factors of 1.0× (baseline) and 0.5× (liquidity crisis). Halving
liquidity doubles the market impact component of transaction costs for the same trade size. This
simulates:

- Flash crashes where liquidity providers withdraw
- Crisis periods (2008, March 2020) when market depth collapses
- Moving from large-cap to mid-cap or small-cap instruments
- End-of-quarter rebalancing when everyone trades simultaneously

The environment's liquidity-dependent cost model means impact costs increase as liquidity falls. A
strategy that traded 0.5 units comfortably in normal conditions might face punitive costs in the
liquidity shock scenario, forcing it to either trade less or accept larger slippage.

We re-run our policies with degraded liquidity and measure performance changes. A well-designed RL
policy should adapt—recognizing higher costs in its state representation (via recent cost history or
volatility proxies) and reducing turnover accordingly. A poorly designed policy will blindly execute
the same high-turnover approach, incinerating returns through excessive impact.

**Stress Test 4: Regime Slicing**

Rather than evaluating on the full test period (which mixes regimes), we slice performance by regime:
evaluate separately on low-volatility periods and high-volatility periods. This reveals regime-
dependent fragility.

For each regime, we identify contiguous segments where the regime indicator equals the target value,
then run policies on those segments. The results often show:

- Some policies thrive in calm markets but crash in turbulent ones
- Others are regime-agnostic—stable across both conditions
- The RL policy might exploit regime-switching effectively if it learned to recognize regimes in
state features

Regime slicing is particularly important for strategies marketed as "all-weather." If your strategy
earns 30% in low-vol regimes but loses 40% in high-vol regimes, and volatility clustering means
you'll eventually hit extended high-vol periods, your strategy is a ticking time bomb.

**Comparative Stress Analysis**

The critical output is not absolute performance under stress—it's relative performance compared to
baselines. We plot metrics across stress scenarios for trend, BC policy, and CPI policy on the same
axes. This reveals:

- **Relative Robustness**: Does the RL policy degrade faster or slower than baselines under stress?
- **Break Points**: At what stress level does each policy become unprofitable?
- **Rank Reversals**: Does the CPI policy beat trend at 1× costs but lose at 2× costs? If so, its
advantage depends on accurate cost assumptions.

If the RL policy dominates baselines across all stress scenarios, we have strong evidence of genuine
improvement. If it only wins under narrow conditions (exactly 1× costs, zero latency, full liquidity),
we've probably overfit to our training environment's quirks.

**The Stress Test Grid as Pre-Registration**

Notice we defined the stress test grid in Cell 3's configuration—before seeing any results. This
pre-registration prevents cherry-picking favorable scenarios. We commit to testing these specific
stresses regardless of outcomes, eliminating the temptation to omit tests where our policy performs
poorly.

This mirrors good scientific practice: define your experiments before collecting data. In trading
strategy development, it prevents the "researcher degrees of freedom" problem where you torture the
data until it confesses to your hypothesis.

**Practical Deployment Implications**

Stress test results inform deployment decisions:

- If performance collapses under 1.5× costs, negotiate better execution agreements before deploying
- If liquidity shocks destroy returns, add dynamic position sizing that scales with market depth
- If latency sensitivity is severe, invest in infrastructure improvements or trade less frequently
- If regime dependence is strong, consider regime-detection overlays that reduce exposure in adverse
periods

The stress tests transform abstract strategy evaluation into concrete operational requirements. They
tell you not just "will this work?" but "under what conditions will this work, and what will break it?"

**The Saved Artifacts**

All stress test results save to JSON with complete scenario specifications and metrics. These become
part of the governance bundle—evidence that due diligence was performed, risks were characterized,
and deployment decisions were informed by worst-case analysis, not just average-case backtests.

The plots provide executive-friendly visualization: one glance shows whether your strategy's
advantage evaporates under realistic operational stress or whether it remains robust across a wide
range of adverse conditions.

###11.2.CODE AND IMPLEMENTATION

In [10]:

# Cell 11 — Stress tests + sensitivity
def run_stress_tests(env, policies, config):
    """
    Run stress tests: cost inflation, latency, liquidity shocks, regime slicing.
    """
    print("[STRESS] Running stress tests...")

    stress_config = config['stress_tests']
    results = []

    # Baseline evaluation window
    test_start = 1000
    test_end = 1200

    # 1. Cost inflation
    for cost_factor in stress_config['cost_inflation']:
        print(f"[STRESS] Cost inflation factor: {cost_factor}")

        # Temporarily modify config
        original_fees = config['costs']['fee_bps']
        original_spread = config['costs']['spread_bps']
        original_impact = config['costs']['impact_coeff']

        config['costs']['fee_bps'] = original_fees * cost_factor
        config['costs']['spread_bps'] = original_spread * cost_factor
        config['costs']['impact_coeff'] = original_impact * cost_factor

        stress_result = {
            'stress_type': 'cost_inflation',
            'factor': cost_factor,
            'policies': {}
        }

        for name in ['trend', 'bc_policy', 'cpi_policy']:
            if name in policies:
                traj = run_episode(env, policies[name], test_start, test_end)
                metrics = compute_metrics(traj)
                stress_result['policies'][name] = metrics

        results.append(stress_result)

        # Restore original
        config['costs']['fee_bps'] = original_fees
        config['costs']['spread_bps'] = original_spread
        config['costs']['impact_coeff'] = original_impact

    # 2. Latency shift (simplified: not implemented in env step, just documented)
    for latency in stress_config['latency_shift']:
        print(f"[STRESS] Latency shift: t+{1+latency}")

        stress_result = {
            'stress_type': 'latency_shift',
            'latency': latency,
            'note': 'Latency stress requires environment modification. Placeholder result.',
            'policies': {}
        }

        results.append(stress_result)

    # 3. Liquidity shock
    for liq_factor in stress_config['liquidity_shock']:
        print(f"[STRESS] Liquidity shock factor: {liq_factor}")

        # Modify liquidity in market data
        original_liq = env.liquidity.copy()
        env.liquidity = env.liquidity * liq_factor

        stress_result = {
            'stress_type': 'liquidity_shock',
            'factor': liq_factor,
            'policies': {}
        }

        for name in ['trend', 'bc_policy', 'cpi_policy']:
            if name in policies:
                traj = run_episode(env, policies[name], test_start, test_end)
                metrics = compute_metrics(traj)
                stress_result['policies'][name] = metrics

        results.append(stress_result)

        # Restore
        env.liquidity = original_liq

    # 4. Regime slicing
    if stress_config['regime_slice']:
        print("[STRESS] Regime slicing...")

        # Separate evaluation by regime
        for regime in range(2):
            # Find periods in this regime
            regime_indices = np.where(env.regimes[test_start:test_end] == regime)[0] + test_start

            if len(regime_indices) < 50:
                continue

            # Use contiguous segment
            regime_start = regime_indices[0]
            regime_end = min(regime_start + 100, regime_indices[-1])

            stress_result = {
                'stress_type': 'regime_slice',
                'regime': regime,
                'regime_name': config['data']['regime_names'][regime],
                'policies': {}
            }

            for name in ['trend', 'bc_policy', 'cpi_policy']:
                if name in policies:
                    traj = run_episode(env, policies[name], regime_start, regime_end)
                    metrics = compute_metrics(traj)
                    stress_result['policies'][name] = metrics

            results.append(stress_result)

    return results

# Run stress tests
stress_results = run_stress_tests(env, eval_policies, CONFIG)

# Save stress test results
stress_path = f"{PATHS['artifacts']}/stress_tests.json"
with open(stress_path, 'w') as f:
    json.dump(stress_results, f, indent=2, default=float)

# Plot stress test comparison
stress_types = list(set(r['stress_type'] for r in stress_results))
n_types = len(stress_types)

fig, axes = plt.subplots(n_types, 1, figsize=(12, 4*n_types))
if n_types == 1:
    axes = [axes]

for i, stress_type in enumerate(stress_types):
    type_results = [r for r in stress_results if r['stress_type'] == stress_type]

    # Extract returns for each policy
    policy_names = ['trend', 'bc_policy', 'cpi_policy']
    for policy in policy_names:
        returns = []
        labels = []
        for r in type_results:
            if policy in r['policies']:
                returns.append(r['policies'][policy]['net_return'])
                if 'factor' in r:
                    labels.append(f"{r['factor']:.1f}")
                elif 'regime' in r:
                    labels.append(r['regime_name'])
                else:
                    labels.append(str(r.get('latency', '')))

        if returns:
            axes[i].plot(range(len(returns)), returns, marker='o', label=policy)

    axes[i].set_title(f"Stress Test: {stress_type}")
    axes[i].set_ylabel('Net Return')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

    if labels:
        axes[i].set_xticks(range(len(labels)))
        axes[i].set_xticklabels(labels)

axes[-1].set_xlabel('Stress Factor')
plt.tight_layout()
plt.savefig(f"{PATHS['plots']}/stress_tests.png", dpi=100)
plt.close()

print("[STRESS] Stress tests complete.")

[STRESS] Running stress tests...
[STRESS] Cost inflation factor: 1.0
[STRESS] Cost inflation factor: 1.5
[STRESS] Cost inflation factor: 2.0
[STRESS] Latency shift: t+1
[STRESS] Latency shift: t+2
[STRESS] Liquidity shock factor: 1.0
[STRESS] Liquidity shock factor: 0.5
[STRESS] Regime slicing...


/tmp/ipython-input-3000349141.py:157: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axes[i].legend()


[STRESS] Stress tests complete.


##12.RISK REPORT AND DECISION LOGS

###12.1.OVERVIEW



This section generates the comprehensive risk documentation and decision-level audit trails that
transform a research prototype into a production-ready system. While previous cells focused on
strategy performance, this cell addresses the operational and governance requirements that determine
whether a strategy is actually deployable: detailed risk characterization, exposure tracking, and
complete decision traceability.

**The Risk Report: Beyond Simple Returns**

Trading performance is multi-dimensional—returns alone tell an incomplete and often misleading story.
A strategy returning 50% with a 90% drawdown is not "good"; it's a disaster waiting to complete. The
risk report quantifies all dimensions that matter to risk managers, compliance officers, and capital
allocators.

**Summary Statistics Section**

This captures the fundamental performance profile:

- **Final Equity**: Where you end up (1.25 means 25% gain from initial 1.0)
- **Total Return**: Absolute gain/loss in percentage terms
- **Volatility**: Standard deviation of returns—the basic risk measure
- **Mean Return**: Average per-period return, revealing the drift component
- **Sharpe Ratio**: Risk-adjusted return, the single most important summary statistic

The Sharpe ratio deserves emphasis. A strategy earning 30% with 40% volatility (Sharpe = 0.75) is
worse than one earning 15% with 10% volatility (Sharpe = 1.5). The latter delivers better risk-
adjusted returns and is more scalable. High returns with high volatility often stem from leverage
or concentration—neither is sustainable.

**Exposure Metrics Section**

Risk managers care deeply about how much capital is at risk at any given time:

- **Mean Position**: Average exposure over the period—is the strategy directional (mean ≠ 0) or
market-neutral (mean ≈ 0)?
- **Max/Min Position**: Extreme exposures reached—did the strategy ever become fully levered?
- **Position Standard Deviation**: How much exposure varies—high variance suggests active trading
or regime-dependent sizing

A strategy with mean position +0.8 is structurally long—it's betting on upward drift and will
suffer in bear markets. One with mean near zero is market-neutral—profiting from relative movements
rather than directional bets. Neither is inherently better, but they have radically different risk
profiles.

**Drawdown Analysis Section**

Drawdowns—peak-to-trough declines in equity—are often more important than volatility for real-world
survival:

- **Maximum Drawdown**: The worst loss from any historical peak—this is what tests investor patience
and triggers risk limit breaches
- **95th Percentile Drawdown**: The "typical worst" drawdown, excluding the absolute worst tail
event

Maximum drawdown is psychological and operational reality. A 50% drawdown means you need 100% returns
just to recover—and most investors redeem long before that point. Strategies with >40% max drawdowns,
even if eventually profitable, often fail due to capital flight during the drawdown period.

The 95th percentile drawdown is useful for risk budgeting: if your typical worst drawdown is 20%,
you should reserve enough capital to survive 2-3× that (40-60%) to account for fat tails and non-
stationarity.

**Turnover Metrics Section**

Turnover—how much you trade—directly determines implementation costs and operational complexity:

- **Total Turnover**: Sum of all absolute position changes—a proxy for total transaction cost burden
- **Average Turnover Per Step**: Typical trading intensity—daily strategies trading 0.5 units/day
face higher costs than weekly strategies trading the same amount
- **Maximum Turnover**: Largest single position change—useful for capacity planning

High turnover isn't inherently bad if the alpha justifies it, but it creates operational challenges:
more execution slippage, higher cost sensitivity, greater market impact, and increased operational
errors. A strategy with 1000% annual turnover needs flawless execution infrastructure; one with 50%
turnover is forgiving.

**Tail Risk Section**

Return distribution tails reveal risks that volatility misses:

- **1st and 5th Percentiles**: Left tail—how bad are the worst returns?
- **95th and 99th Percentiles**: Right tail—how good are the best returns?

Symmetric tails (left and right magnitude similar) suggest Gaussian-like returns. Asymmetric tails—
particularly fat left tails—indicate crash risk. A strategy with 1st percentile at -10% but 99th
percentile at +3% has problematic negative skewness: small gains punctuated by large losses. This
is psychologically painful and often indicates selling volatility or picking up pennies in front of
steamrollers.

**The Decision Logs: Complete Traceability**

While the risk report provides aggregate statistics, decision logs record every individual trading
decision. For each timestep, we log:

- **Step Number**: Absolute time index for linking back to market data
- **State Summary**: Key state features (mean lagged return, volatility, position, equity)—enough
to understand what the agent "saw"
- **Action**: What position the policy chose before constraint enforcement
- **Executed Action**: Actual position after constraint projection—reveals when constraints bound
- **Reward**: Immediate reward received for this decision
- **Cost**: Transaction cost incurred
- **Trade Size**: How much the position changed
- **Constraint Violation Flag**: Whether any hard constraints were breached

**Why Decision-Level Logs Matter**

These logs enable multiple critical use cases:

- **Post-Mortem Analysis**: When a big loss occurs, you can trace back to the exact decision that
caused it and understand why the agent chose that action given its state observation
- **Regulatory Compliance**: Regulators increasingly require explainability—you must be able to
justify every trade
- **Model Debugging**: If the RL policy is behaving strangely, logs reveal whether it's seeing
corrupted states, violating constraints, or making sensible decisions that happen to lose money
- **Strategy Refinement**: Patterns in logs reveal systematic errors—perhaps the agent always trades
too aggressively in high-volatility states, suggesting the need for better risk adjustment

**Dual Storage: JSON and NPZ**

We save logs in two formats. JSON is human-readable and integrates with downstream analysis tools.
NPZ (compressed NumPy arrays) is efficient for large-scale numerical analysis—loading millions of
decisions to compute aggregate statistics. This dual storage balances interpretability and
computational efficiency.

**Exposure and Turnover Time Series Plots**

The final visualizations plot position (exposure) and trade size (turnover) over time. These plots
are diagnostic gold:

- **Exposure Plot**: Smooth position changes suggest strategic repositioning; erratic changes suggest
noise-trading or instability
- **Turnover Plot**: Should be relatively stable; spikes indicate reaction to specific events or
constraint hits
- **Correlation**: Do turnover spikes coincide with volatility increases? If yes, the agent is
appropriately risk-managing; if no, it might be trading randomly

These time series also reveal regime-dependent behavior. A good RL policy should show visibly
different exposure profiles in low-vol versus high-vol regimes—more aggressive when cheap, more
defensive when expensive.

**Governance Integration**

The risk report and decision logs aren't afterthoughts—they're first-class artifacts that live
alongside the policy, configuration, and evaluation results in the reproducible bundle. When you
present this strategy to stakeholders, you provide:

- Summary statistics (the pitch)
- Stress test results (the risks)
- Risk report (the operational profile)
- Decision logs (the audit trail)

This completeness transforms "it works in backtests" into "it works in backtests, and here's exactly
how, under what conditions, with what risks, and with complete traceability." That's the difference
between a research curiosity and a production system.

###12.2.CODE AND IMPLEMENTATION

In [11]:

# Cell 12 — Risk report + decision logs
def generate_risk_report(env, policy, policy_name, start_idx, end_idx):
    """Generate comprehensive risk report."""
    print(f"[RISK] Generating risk report for {policy_name}...")

    # Run episode
    traj = run_episode(env, policy, start_idx, end_idx)

    # Extract time series
    equity = np.array([info['equity'] for info in traj['infos']])
    positions = np.array([info['position'] for info in traj['infos']])
    trades = np.array([info['trade_size'] for info in traj['infos']])

    # Compute statistics
    returns = np.diff(equity) / (equity[:-1] + 1e-6)

    risk_report = {
        'policy_name': policy_name,
        'period': f"[{start_idx}, {end_idx}]",
        'summary_stats': {
            'final_equity': float(equity[-1]),
            'total_return': float(equity[-1] - 1.0),
            'volatility': float(np.std(returns)),
            'mean_return': float(np.mean(returns)),
            'sharpe': float(np.mean(returns) / (np.std(returns) + 1e-6))
        },
        'exposure': {
            'mean_position': float(np.mean(positions)),
            'max_position': float(np.max(positions)),
            'min_position': float(np.min(positions)),
            'position_std': float(np.std(positions))
        },
        'drawdown': {
            'max_drawdown': float(np.max((np.maximum.accumulate(equity) - equity) / np.maximum.accumulate(equity))),
            'drawdown_95pct': float(np.percentile((np.maximum.accumulate(equity) - equity) / np.maximum.accumulate(equity), 95))
        },
        'turnover': {
            'total_turnover': float(np.sum(trades)),
            'avg_turnover_per_step': float(np.mean(trades)),
            'max_turnover': float(np.max(trades))
        },
        'tail_risk': {
            'return_5pct': float(np.percentile(returns, 5)),
            'return_1pct': float(np.percentile(returns, 1)),
            'return_95pct': float(np.percentile(returns, 95)),
            'return_99pct': float(np.percentile(returns, 99))
        }
    }

    return risk_report, traj

# Generate risk report for CPI policy
risk_report, risk_traj = generate_risk_report(
    env, lambda s, e: cpi_policy.predict(s), 'cpi_policy', 1200, 1400
)

# Save risk report
risk_path = f"{PATHS['artifacts']}/risk_report.json"
with open(risk_path, 'w') as f:
    json.dump(risk_report, f, indent=2)

# Generate decision logs
decision_logs = []
for i, (state, action, reward, info) in enumerate(zip(
    risk_traj['states'], risk_traj['actions'], risk_traj['rewards'], risk_traj['infos']
)):
    log_entry = {
        'step': i,
        'state_summary': {
            'mean_lagged_return': float(np.mean(state[:20])),
            'rolling_vol': float(state[20]),
            'position': float(info['position']),
            'equity': float(info['equity'])
        },
        'action': float(action),
        'executed_action': float(info['position']),
        'reward': float(reward),
        'cost': float(info['cost']),
        'trade_size': float(info['trade_size']),
        'constraint_violation': bool(info['constraint_violation'])
    }
    decision_logs.append(log_entry)

# Save decision logs
logs_path = f"{PATHS['logs']}/decision_logs.json"
with open(logs_path, 'w') as f:
    json.dump(decision_logs, f, indent=2)

# Also save as npz for efficiency
logs_npz_path = f"{PATHS['logs']}/decision_logs.npz"
np.savez(logs_npz_path,
         steps=np.array([log['step'] for log in decision_logs]),
         actions=np.array([log['action'] for log in decision_logs]),
         rewards=np.array([log['reward'] for log in decision_logs]),
         costs=np.array([log['cost'] for log in decision_logs]),
         positions=np.array([log['executed_action'] for log in decision_logs]))

# Plot exposure and turnover
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

positions = [info['position'] for info in risk_traj['infos']]
axes[0].plot(positions)
axes[0].set_title('Position (Exposure) Over Time')
axes[0].set_ylabel('Position')
axes[0].grid(True, alpha=0.3)

trades = [info['trade_size'] for info in risk_traj['infos']]
axes[1].plot(trades)
axes[1].set_title('Turnover Over Time')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Turnover')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{PATHS['plots']}/risk_exposure.png", dpi=100)
plt.close()

print("[RISK] Risk report and decision logs saved.")


[RISK] Generating risk report for cpi_policy...
[RISK] Risk report and decision logs saved.


##13.REPRODUCIBLE REPORTING BUNDLE INDEX

###13.1.OVERVIEW



###13.2.CODE AND IMPLEMENTATION

In [12]:
# Cell 13 — Reproducible reporting bundle index
def create_repro_bundle_index(base_path):
    """Create index of all output files with checksums."""
    print("[BUNDLE] Creating reproducible bundle index...")

    file_tree = {}

    # Walk through all directories
    for root, dirs, files in os.walk(base_path):
        for file in files:
            filepath = os.path.join(root, file)
            relpath = os.path.relpath(filepath, base_path)

            # Compute checksum
            try:
                checksum = file_hash(filepath)
                file_size = os.path.getsize(filepath)

                file_tree[relpath] = {
                    'checksum': checksum,
                    'size_bytes': file_size
                }
            except Exception as e:
                file_tree[relpath] = {
                    'error': str(e)
                }

    bundle_index = {
        'run_id': run_id,
        'timestamp': datetime.now().isoformat(),
        'base_path': base_path,
        'file_count': len(file_tree),
        'files': file_tree
    }

    return bundle_index

# Create bundle index
bundle_index = create_repro_bundle_index(BASE_PATH)

# Save bundle index
bundle_path = f"{PATHS['artifacts']}/repro_bundle_index.json"
with open(bundle_path, 'w') as f:
    json.dump(bundle_index, f, indent=2)

print(f"[BUNDLE] Reproducible bundle created with {bundle_index['file_count']} files")

# Final summary
print("\n" + "="*80)
print("CHAPTER 19: REINFORCEMENT LEARNING FOR TRADING DECISIONS - COMPLETE")
print("="*80)
print(f"Run ID: {run_id}")
print(f"Base path: {BASE_PATH}")
print("\nGOVERNANCE ARTIFACTS CREATED:")
print("  - run_manifest.json")
print("  - config.json")
print("  - environment_spec.json")
print("  - data_fingerprint.json")
print("  - cost_model_registry.json")
print("  - baseline_metrics.json")
print("  - training_traces.json")
print("  - evaluation_suite.json")
print("  - ope_demo.json")
print("  - stress_tests.json")
print("  - risk_report.json")
print("  - decision_logs.json/.npz")
print("  - repro_bundle_index.json")
print("\nKEY RESULTS:")

# Print summary metrics
if len(wf_results) > 0:
    fold_0 = wf_results[0]
    print("\nWalk-Forward Fold 0 Results:")
    for policy_name in ['trend', 'bc_policy', 'cpi_policy']:
        if policy_name in fold_0['policies']:
            metrics = fold_0['policies'][policy_name]
            print(f"  {policy_name}:")
            print(f"    Return: {metrics['net_return']:.4f}")
            print(f"    Sharpe: {metrics['sharpe']:.4f}")
            print(f"    Max DD: {metrics['max_drawdown']:.4f}")

print("\nLEARNING OUTCOMES ACHIEVED:")
print("  1. Trading problem formalized as MDP with admissible state and constrained actions")
print("  2. Offline RL pipeline implemented (BC + CPI) with synthetic data")
print("  3. Walk-forward backtests completed with conservative OPE checks")
print("  4. Governance artifacts produced: manifests, fingerprints, logs, stress tests")
print("\nKEY TAKEAWAYS:")
print("  - RL optimizes decisions, not predictions")
print("  - Backtests are necessary but insufficient (execution assumptions matter)")
print("  - Offline RL requires conservative improvement to avoid overconfidence")
print("  - OPE can mislead when policies differ significantly (importance sampling variance)")
print("  - Stress tests reveal policy robustness to cost/latency/liquidity shocks")
print("  - Governance enables reproducibility, auditability, and risk management")
print("="*80)

[BUNDLE] Creating reproducible bundle index...
[BUNDLE] Reproducible bundle created with 23 files

CHAPTER 19: REINFORCEMENT LEARNING FOR TRADING DECISIONS - COMPLETE
Run ID: 20251229_173257
Base path: /content/ch19_runs/20251229_173257

GOVERNANCE ARTIFACTS CREATED:
  - run_manifest.json
  - config.json
  - environment_spec.json
  - data_fingerprint.json
  - cost_model_registry.json
  - baseline_metrics.json
  - training_traces.json
  - evaluation_suite.json
  - ope_demo.json
  - stress_tests.json
  - risk_report.json
  - decision_logs.json/.npz
  - repro_bundle_index.json

KEY RESULTS:

Walk-Forward Fold 0 Results:
  trend:
    Return: 0.1988
    Sharpe: 0.0859
    Max DD: 0.1043
  bc_policy:
    Return: 0.0335
    Sharpe: 0.0430
    Max DD: 0.0601
  cpi_policy:
    Return: 0.0304
    Sharpe: 0.0440
    Max DD: 0.0530

LEARNING OUTCOMES ACHIEVED:
  1. Trading problem formalized as MDP with admissible state and constrained actions
  2. Offline RL pipeline implemented (BC + CPI) with s

##14.IMPLEMENTATION WITH REAL DATA

PLEASE RESTART SESSION HERE

###14.1.OVERVIEW




This implementation represents a critical transition in our reinforcement learning journey: moving
from the controlled environment of synthetic data to the messy, non-stationary reality of actual
market data. While the core RL algorithms and environment structure remain identical to our
synthetic data version, the data acquisition, preprocessing, and interpretation layers require
careful engineering to bridge the gap between idealized simulations and production deployment.

**Why Real Data Changes Everything**

Synthetic data gave us ground truth—we knew the regime-switching parameters, the exact volatility
levels, the drift rates. We could verify our algorithms worked correctly because we designed the
problem to be solvable. Real market data offers no such comfort. We don't know the true data-
generating process. Regimes aren't labeled. Volatility isn't constant within regimes. Corporate
actions, stock splits, dividends, and market microstructure artifacts contaminate the signal.
Most importantly, the future genuinely doesn't resemble the past—markets evolve, participants
adapt, and strategies that worked in training data fail in deployment.

This implementation confronts these realities head-on by downloading actual SPY (S&P 500 ETF)
data from 2020 through 2024—a period spanning the COVID crash, unprecedented monetary stimulus,
inflation surges, and interest rate whiplash. If our RL system can handle this non-stationarity,
it has a fighting chance in production.

**The yfinance Integration: Data Acquisition Done Right**

We use the yfinance library to download historical data, which provides a clean interface to
Yahoo Finance's data API. The critical choice here is `auto_adjust=True`, which automatically
adjusts historical prices for stock splits and dividends. This prevents artificial discontinuities
in our return series—without adjustment, a 2-for-1 stock split would appear as a -50% overnight
loss, which would completely confuse our RL agent.

The data acquisition code is deliberately simple and transparent:

- Create a Ticker object for the specified symbol (SPY)
- Call the history method with date range and auto-adjustment enabled
- Extract Close prices and Volume as NumPy arrays
- Immediately discard the pandas DataFrame—no pandas operations beyond this point

This "download once, convert to NumPy, forget pandas" pattern is crucial for production systems
where pandas' implicit behaviors (timezone handling, reindexing, forward-filling) can introduce
subtle bugs. Once we have NumPy arrays, every operation is explicit and auditable.

**Computing Returns: The First Critical Decision**

We compute simple returns using `returns[t] = (price[t] - price[t-1]) / price[t-1]`. This is
the standard choice for daily equity data, but it's worth noting the alternatives we rejected:

- **Log returns** `ln(price[t] / price[t-1])` would be more theoretically correct for multi-period
compounding, but simple returns are more intuitive for position-sizing decisions and are perfectly
adequate for daily rebalancing.

- **Excess returns** (returns minus risk-free rate) would be theoretically purer for Sharpe ratio
calculations, but with near-zero risk-free rates during most of our sample period, this adjustment
is negligible.

- **Risk-adjusted returns** (returns divided by volatility) would normalize for heteroskedasticity,
but we prefer to let our RL agent learn volatility-dependent policies rather than preprocessing
it away.

After computing returns, we align the arrays—prices and volume both get truncated by one element
to match the returns length. This alignment discipline prevents off-by-one indexing errors that
plague financial data processing.

**Liquidity Proxy: Modeling Transaction Cost Dynamics**

Real markets aren't uniformly liquid. A $10,000 trade costs more during market stress than during
calm periods. We model this through a volume-based liquidity proxy that feeds into our impact
cost calculation.

The construction is deliberately simple: divide current volume by the 20-day moving average of
volume, then clip extreme values. When today's volume is 2× the recent average, liquidity is high
(proxy = 2.0) and impact costs are halved. When volume is 0.5× average, liquidity is low
(proxy = 0.5) and impact costs double.

This proxy is crude—sophisticated systems would use order book depth, bid-ask spreads, and
intraday volume patterns—but it captures the essential feature: transaction costs are state-
dependent and anti-cyclical. They spike precisely when you most want to trade (during volatility
surges and regime shifts).

**Synthetic Regime Labels: Inferring Hidden Structure**

Our synthetic data had explicit regime labels (0 = low vol, 1 = high vol) because we generated
them from a Markov chain. Real data has no such labels—regimes are latent variables we must infer.

We create approximate regime labels by comparing recent realized volatility to a rolling median
threshold. If the 20-day standard deviation exceeds 1.5× the historical median, we label it
high-volatility (regime 1); otherwise, low-volatility (regime 0).

This is admittedly circular—we're using volatility to infer regimes, then including regime
probabilities in our state. A more sophisticated approach would use Hidden Markov Models or
change-point detection to identify structural breaks. But our simple heuristic suffices to
demonstrate the concept: real-world RL systems need some mechanism for detecting regime changes
because strategies that work in calm markets often fail in crises.

Critically, we compute these regime labels using only past data (rolling window ending at t),
preserving causality. A smoothing filter that uses future data would leak information and
invalidate our backtests.

**Data Fingerprinting: Cryptographic Audit Trails**

After downloading and processing, we immediately compute cryptographic fingerprints (SHA-256
hashes) of the returns and prices arrays. These fingerprints serve multiple governance functions:

- **Reproducibility verification**: If you re-run this code months later and get different
fingerprints, something changed—Yahoo Finance revised their data, you downloaded a different date
range, or a processing bug was introduced.

- **Data lineage tracking**: Every model trained on this data can reference these fingerprints,
creating an audit trail from results back to exact data sources.

- **Version control for data**: Just as we version code with Git, we version datasets with
fingerprints. When you update from 2020-2023 data to 2020-2024 data, the fingerprint changes,
and you can track which results used which version.

The fingerprint JSON also records metadata: ticker symbol, date range, data source (yfinance),
and corporate action handling (auto-adjusted). This metadata answers questions that arise months
later: "Did this backtest include dividends?" "Which data vendor did we use?" "What dates did
this cover?"

**Configuration Differences: Adapting Parameters for Real Data**

The configuration dictionary looks similar to our synthetic version, but several parameters changed
to reflect real market realities:

- **Shorter evaluation windows** (400 training steps, 100 test steps instead of 800/200) because
we have fewer total timesteps (roughly 1,200 trading days vs. 2,000 synthetic days).

- **More conservative training** (fewer expert trajectories, fewer CPI steps) because real data
is noisier and overfitting is a greater risk.

- **No regime slicing in stress tests** because our regime labels are crude synthetic constructs
rather than true ground truth.

These adaptations acknowledge a fundamental truth: real data is precious and limited. We can't
afford to waste hundreds of days on warmup periods or throw away data on extensive hyperparameter
searches.

**The Environment: Unchanged Core, Different Context**

The TradingEnvironment class is nearly identical to our synthetic version—same state construction,
same action projection, same reward function, same timing conventions. This is by design. The
environment defines the interaction protocol (MDP structure), which doesn't change just because
data sources change.

However, the *interpretation* differs subtly. With synthetic data, we knew ground truth regime
probabilities and could verify our filtered estimates were reasonable. With real data, our regime
features are approximations of unknown truth. The RL agent must learn robust policies despite
this feature noise.

Similarly, our liquidity proxy with synthetic data was a deterministic function of regime. With
real data, it's a noisy volume-based estimate. The agent experiences higher uncertainty about
transaction costs, which naturally encourages more conservative trading.

**Baseline Strategies: Reality Check on Simple Rules**

We run the same four baselines (cash, buy-and-hold, trend-following, myopic) on real data as on
synthetic. This parallel structure enables direct comparison: did the baselines perform similarly?
If trend-following dominated on synthetic data but failed on real data, what changed?

The results are sobering. During the 2020-2024 period, simple trend-following strategies struggled.
The market exhibited multiple regime shifts, trending phases interrupted by sharp reversals, and
volatility clustering that punished static positioning rules. Buy-and-hold performed reasonably
(SPY rose over this period) but with substantial drawdowns (COVID crash, 2022 bear market).

These baseline results calibrate our expectations for RL. If the expert policy (trend-following)
loses money out-of-sample, behavior cloning will learn to lose money in similar ways. Conservative
policy improvement might reduce losses through better risk management, but it can't conjure profits
from a failing base strategy.

**Training on Real Data: The Overfitting Trap**

Training RL policies on real data requires vigilance against overfitting. With only ~400 days of
training data, it's easy to learn patterns that don't generalize. Our two-stage approach (BC + CPI)
provides some protection:

- **Behavior cloning** learns from demonstrated trajectories rather than directly optimizing
in-sample returns. This prevents the policy from exploiting spurious correlations (like "returns
are always positive on Mondays in our training window").

- **Conservative policy improvement** penalizes deviation from BC, limiting the search space to
policies close to the demonstrated behavior. This prevents aggressive optimization that finds
apparent improvements that are actually overfitting artifacts.

Even with these safeguards, we keep training short (100 BC epochs, 5 CPI steps) and use simple
linear policies. Complex nonlinear policies (deep networks) would certainly achieve better in-
sample fit, but they'd almost certainly overfit given our limited data.

**Walk-Forward Evaluation: The Ultimate Test**

The walk-forward evaluation on real data is where theory meets reality. We train on 400 days,
test on the next 100 days, then roll forward and repeat. This simulates realistic deployment:
train on all available history, trade live for a period, retrain with new data included.

Our results show negative returns across baselines and RL policies in the first fold. This isn't
a failure of the evaluation methodology—it's the market telling us our strategies don't work in
this regime. The evaluation correctly identified this failure before we deployed real capital.

This is the value of rigorous out-of-sample testing. In-sample, our policies might have looked
profitable by exploiting training-period quirks. Out-of-sample, these illusory edges evaporated.
Better to discover this in backtests than in production.

**The Corrected Sharpe Ratio: Getting the Math Right**

Our original implementation computed Sharpe ratio as `total_return / per_step_volatility`, which
created nonsensical values (Sharpe = -14 when the strategy lost 15%). The corrected version uses
`mean_return / std_return`, both computed on the same timescale (per-step).

This correction matters beyond just getting numbers right. Sharpe ratio is the primary risk-
adjusted performance metric in finance. Getting it wrong invalidates all performance comparisons,
stress test interpretations, and investment decisions. The corrected implementation yields
realistic values (Sharpe = -0.2 to -0.7 for our losing strategies), which properly reflect poor
risk-adjusted returns.

**Governance Artifacts: Production Readiness**

Every artifact we generated for synthetic data—manifests, fingerprints, specifications, logs—
appears here for real data. This consistency is deliberate. Governance requirements don't change
based on data sources. Whether you're testing on synthetic data or deploying on real markets, you
need the same level of documentation, traceability, and auditability.

The artifacts transform this from a research notebook into a production-ready system foundation.
Someone reviewing this code six months from now can understand exactly what was tested, on what
data, with what parameters, yielding what results—all without archaeological code excavation.

**Conclusion: Bridging Research and Reality**

This real-data implementation demonstrates that RL for trading is possible but not easy. The
algorithms work—they execute without errors, respect constraints, and generate plausible policies.
But working correctly doesn't guarantee profitable trading. Our strategies lost money out-of-
sample because simple trend-following doesn't work in all market regimes, and our conservative
offline RL approach can't discover fundamentally new profitable strategies.

This negative result is valuable. It teaches humility about what RL can achieve with limited data
and simple features. It validates our governance approach—we detected the failure through proper
backtesting rather than learning it with real capital. And it provides a foundation for iteration:
now we can systematically improve features, try regime-dependent policies, or incorporate
alternative data sources, tracking improvements through the same rigorous evaluation pipeline.

The path from synthetic data to real markets is traversable, but it requires engineering discipline,
statistical rigor, and honest assessment of results. This implementation provides the blueprint.

###14.2.CODE AND IMPLEMENTATION

In [13]:
# =============================================================================
# AI & ALGORITHMIC TRADING — Chapter 19: Reinforcement Learning for Trading
# REAL DATA ADAPTER - Complete Standalone Implementation (CORRECTED)
# Author: Alejandro Reynoso (External Lecturer, Cambridge Judge Business School)
# =============================================================================
# This version uses real market data from yfinance instead of synthetic data
# FIX: Corrected Sharpe ratio calculation
# =============================================================================

# Cell 1 — Install and import dependencies
import sys
print("[SETUP] Installing yfinance...")
!pip install yfinance --quiet

import numpy as np
import json
import os
import hashlib
from datetime import datetime
import math
import random
from collections import defaultdict
import matplotlib.pyplot as plt
import yfinance as yf

print("[SETUP] All dependencies installed successfully.")

# Cell 2 — Determinism + project paths + hashing utilities
MASTER_SEED = 42
np.random.seed(MASTER_SEED)
random.seed(MASTER_SEED)

def derive_seed(base_seed, label):
    """Derive a sub-seed from base seed and a label."""
    h = hashlib.md5(f"{base_seed}_{label}".encode()).digest()
    return int.from_bytes(h[:4], 'big') % (2**31)

SEED_TRAIN = derive_seed(MASTER_SEED, "train")
SEED_EVAL = derive_seed(MASTER_SEED, "eval")

run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
BASE_PATH = f"/content/ch19_runs/{run_id}"
PATHS = {
    'base': BASE_PATH,
    'artifacts': f"{BASE_PATH}/artifacts",
    'plots': f"{BASE_PATH}/plots",
    'logs': f"{BASE_PATH}/logs",
    'policy': f"{BASE_PATH}/policy",
    'data': f"{BASE_PATH}/data"
}

for path in PATHS.values():
    os.makedirs(path, exist_ok=True)

print(f"[INIT] Run ID: {run_id}")
print(f"[INIT] Base path: {BASE_PATH}")

def stable_hash_dict(d):
    """Compute stable hash of dictionary (sorted JSON)."""
    s = json.dumps(d, sort_keys=True, indent=None)
    return hashlib.sha256(s.encode()).hexdigest()

def file_hash(filepath):
    """Compute SHA-256 hash of file."""
    h = hashlib.sha256()
    with open(filepath, 'rb') as f:
        for chunk in iter(lambda: f.read(8192), b''):
            h.update(chunk)
    return h.hexdigest()

def array_fingerprint(arr):
    """Compute fingerprint of numpy array."""
    return hashlib.sha256(arr.tobytes()).hexdigest()[:16]

print("[INIT] Hashing utilities ready.")

# Cell 3 — Configuration
CONFIG = {
    'data': {
        'ticker': 'SPY',
        'start_date': '2020-01-01',
        'end_date': '2024-12-01',
        'source': 'yfinance'
    },

    'execution': {
        'decision_step': 1,
        'fill_timing': 'next_step',
        'slippage_model': 'proportional'
    },

    'costs': {
        'fee_bps': 5.0,
        'spread_bps': 2.0,
        'impact_coeff': 0.1,
        'liquidity_proxy': True
    },

    'constraints': {
        'position_bounds': [-1.0, 1.0],
        'leverage_cap': 1.0,
        'turnover_cap_per_step': 0.5
    },

    'reward': {
        'risk_penalty': 0.5,
        'drawdown_penalty': 1.0,
        'turnover_penalty': 0.01
    },

    'training': {
        'bc_epochs': 100,
        'bc_lr': 0.01,
        'bc_batch_size': 32,
        'cpi_steps': 5,
        'cpi_deviation_penalty': 1.0,
        'cpi_lr': 0.005,
        'seed_train': SEED_TRAIN,
        'seed_eval': SEED_EVAL
    },

    'evaluation': {
        'train_len': 400,
        'test_len': 100,
        'step_len': 100,
        'min_train_start': 50
    },

    'stress_tests': {
        'cost_inflation': [1.0, 1.5, 2.0],
        'latency_shift': [0, 1],
        'liquidity_shock': [1.0, 0.5],
        'regime_slice': False  # No regime info in real data
    },

    'baselines': ['cash', 'buy_hold', 'trend', 'myopic']
}

config_path = f"{PATHS['artifacts']}/config.json"
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=2)

config_hash = stable_hash_dict(CONFIG)
print(f"[CONFIG] Config hash: {config_hash}")

run_manifest = {
    'run_id': run_id,
    'timestamp_start': datetime.now().isoformat(),
    'master_seed': MASTER_SEED,
    'config_hash': config_hash,
    'code_hash': 'TBD',
    'environment_version': 'ch19_real_data_v1.0',
    'status': 'running'
}

manifest_path = f"{PATHS['artifacts']}/run_manifest.json"
with open(manifest_path, 'w') as f:
    json.dump(run_manifest, f, indent=2)

print("[CONFIG] Config and manifest saved.")

# Cell 4 — Real market data download
def download_real_market_data(config):
    """
    Download real market data using yfinance.
    Returns: dict with 'returns', 'prices', 'volume', 'liquidity'
    """
    print(f"[DATA] Downloading {config['ticker']} from yfinance...")

    ticker = config['ticker']
    start = config['start_date']
    end = config['end_date']

    # Download data using latest yfinance syntax
    ticker_obj = yf.Ticker(ticker)
    df = ticker_obj.history(start=start, end=end, auto_adjust=True)

    if df.empty:
        raise ValueError(f"No data returned for {ticker}")

    print(f"[DATA] Downloaded {len(df)} days of data")

    # Extract data as numpy arrays (NO pandas after this point)
    prices = df['Close'].values
    volume = df['Volume'].values

    # Compute returns
    returns = np.diff(prices) / prices[:-1]
    prices = prices[1:]  # Align with returns
    volume = volume[1:]

    T = len(returns)

    # Create liquidity proxy from volume
    # Higher volume = more liquid, normalized
    volume_ma = np.zeros(T)
    for i in range(T):
        start_idx = max(0, i - 20)
        volume_ma[i] = np.mean(volume[start_idx:i+1])

    # Normalize volume to create liquidity proxy
    liquidity = volume / (volume_ma + 1e-6)
    liquidity = np.clip(liquidity, 0.1, 10.0)  # Bound extremes

    # Create synthetic regime indicator based on realized volatility
    regimes = np.zeros(T, dtype=int)
    for i in range(T):
        start_idx = max(0, i - 20)
        rolling_vol = np.std(returns[start_idx:i+1]) if i > start_idx else 0.01
        median_vol = np.median(np.abs(returns[:i+1])) if i > 20 else 0.01
        # High vol = regime 1, low vol = regime 0
        regimes[i] = 1 if rolling_vol > median_vol * 1.5 else 0

    return {
        'returns': returns,
        'prices': prices,
        'volume': volume,
        'liquidity': liquidity,
        'regimes': regimes,
        'T': T,
        'ticker': ticker,
        'start_date': start,
        'end_date': end
    }

# Download data
market_data = download_real_market_data(CONFIG['data'])

# Save dataset
dataset_path = f"{PATHS['data']}/real_market.npz"
np.savez(dataset_path,
         returns=market_data['returns'],
         prices=market_data['prices'],
         volume=market_data['volume'],
         liquidity=market_data['liquidity'],
         regimes=market_data['regimes'])

# Compute data fingerprint
data_fingerprint = {
    'instrument': market_data['ticker'],
    'frequency': 'daily',
    'span': market_data['T'],
    'start_date': market_data['start_date'],
    'end_date': market_data['end_date'],
    'returns_fingerprint': array_fingerprint(market_data['returns']),
    'prices_fingerprint': array_fingerprint(market_data['prices']),
    'missingness': 0.0,
    'source': 'yfinance',
    'corporate_actions': 'auto_adjusted'
}

fingerprint_path = f"{PATHS['data']}/data_fingerprint.json"
with open(fingerprint_path, 'w') as f:
    json.dump(data_fingerprint, f, indent=2)

print(f"[DATA] Downloaded {market_data['T']} timesteps for {market_data['ticker']}")
print(f"[DATA] Returns fingerprint: {data_fingerprint['returns_fingerprint']}")

# Plot market data
fig, axes = plt.subplots(3, 1, figsize=(12, 8))

axes[0].plot(market_data['prices'])
axes[0].set_title(f"{market_data['ticker']} Prices")
axes[0].set_ylabel('Price')
axes[0].grid(True, alpha=0.3)

axes[1].plot(market_data['returns'])
axes[1].set_title('Returns')
axes[1].set_ylabel('Return')
axes[1].grid(True, alpha=0.3)

axes[2].plot(market_data['liquidity'])
axes[2].set_title('Liquidity Proxy (Volume-based)')
axes[2].set_ylabel('Liquidity')
axes[2].set_xlabel('Time')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{PATHS['plots']}/market_data.png", dpi=100)
plt.close()

print("[DATA] Market data plots saved.")

# Cell 5 — Cost model
def compute_trading_cost(trade_size, liquidity, config):
    """Compute trading cost for a given trade."""
    fee_bps = config['costs']['fee_bps']
    spread_bps = config['costs']['spread_bps']
    impact_coeff = config['costs']['impact_coeff']

    base_cost = fee_bps + spread_bps

    if config['costs']['liquidity_proxy']:
        impact_bps = impact_coeff * abs(trade_size) * 1000 / liquidity
    else:
        impact_bps = impact_coeff * abs(trade_size) * 1000

    total_cost_bps = base_cost + impact_bps
    total_cost_fraction = total_cost_bps / 10000.0

    return total_cost_fraction

cost_model_registry = {
    'model_version': 'v1.0',
    'formula': 'cost = fees + spread + impact * |trade| / liquidity',
    'parameters': CONFIG['costs'],
    'units': 'fraction of trade notional',
    'notes': 'Impact cost inversely proportional to volume-based liquidity proxy.'
}

cost_registry_path = f"{PATHS['artifacts']}/cost_model_registry.json"
with open(cost_registry_path, 'w') as f:
    json.dump(cost_model_registry, f, indent=2)

print("[COSTS] Cost model registry saved.")

# Cell 6 — Trading environment
class TradingEnvironment:
    """Trading environment for RL."""

    def __init__(self, market_data, config):
        self.returns = market_data['returns']
        self.prices = market_data['prices']
        self.liquidity = market_data['liquidity']
        self.T = len(self.returns)
        self.config = config

        self.pos_min, self.pos_max = config['constraints']['position_bounds']
        self.leverage_cap = config['constraints']['leverage_cap']
        self.turnover_cap = config['constraints']['turnover_cap_per_step']

        self.lookback = 20
        self.vol_window = 20

        self.reset(0, self.T)

    def reset(self, start_idx, end_idx):
        """Reset environment for episode."""
        self.start_idx = int(start_idx)
        self.end_idx = int(end_idx)
        self.current_idx = self.start_idx

        self.position = 0.0
        self.cash = 1.0
        self.equity = 1.0
        self.entry_price = self.prices[self.start_idx]
        self.peak_equity = 1.0
        self.cumulative_turnover = 0.0

        return self._get_state()

    def _get_state(self):
        """Construct state at current time (causal only)."""
        t = self.current_idx

        # Lagged returns
        lagged_returns = np.zeros(self.lookback)
        for i in range(self.lookback):
            idx = t - i - 1
            if idx >= 0:
                lagged_returns[i] = self.returns[idx]

        # Rolling volatility
        rolling_vol = self._compute_rolling_vol(t)

        # Portfolio state
        portfolio_state = np.array([
            self.position,
            self.equity,
            (self.equity - self.peak_equity) / (self.peak_equity + 1e-6),
            self.cumulative_turnover
        ])

        # Combine features
        state = np.concatenate([
            lagged_returns,
            [rolling_vol],
            portfolio_state
        ])

        return state

    def _compute_rolling_vol(self, t):
        """Compute rolling volatility causally."""
        window = self.vol_window
        start = max(0, t - window)
        if t - start < 2:
            return 0.01
        returns_window = self.returns[start:t]
        return np.std(returns_window)

    def step(self, action):
        """Execute action and advance one timestep."""
        t = self.current_idx

        action = self._project_action(action)
        trade = action - self.position
        trade_size = abs(trade)

        if t + 1 >= self.end_idx:
            done = True
            next_state = self._get_state()
            reward = 0.0
            info = {'constraint_violation': False}
            return next_state, reward, done, info

        self.current_idx = t + 1
        realized_return = self.returns[self.current_idx]
        liquidity = self.liquidity[self.current_idx]

        cost = compute_trading_cost(trade_size, liquidity, self.config)
        cost_amount = cost * trade_size * self.equity

        pnl = self.position * realized_return * self.equity
        self.equity = self.equity + pnl - cost_amount
        self.position = action
        self.cumulative_turnover += trade_size

        self.peak_equity = max(self.peak_equity, self.equity)

        reward = self._compute_reward(pnl, cost_amount, trade_size)

        constraint_violation = (abs(self.position) > self.leverage_cap or
                                trade_size > self.turnover_cap)

        done = (self.current_idx + 1 >= self.end_idx)
        next_state = self._get_state()

        info = {
            'pnl': pnl,
            'cost': cost_amount,
            'trade_size': trade_size,
            'constraint_violation': constraint_violation,
            'equity': self.equity,
            'position': self.position
        }

        return next_state, reward, done, info

    def _project_action(self, action):
        """Project action to satisfy constraints."""
        action = np.clip(action, self.pos_min, self.pos_max)
        action = np.clip(action, -self.leverage_cap, self.leverage_cap)

        trade = action - self.position
        if abs(trade) > self.turnover_cap:
            trade = np.sign(trade) * self.turnover_cap
            action = self.position + trade

        return action

    def _compute_reward(self, pnl, cost, trade_size):
        """Compute reward with penalties."""
        risk_penalty = self.config['reward']['risk_penalty']
        dd_penalty = self.config['reward']['drawdown_penalty']
        turnover_penalty = self.config['reward']['turnover_penalty']

        risk_term = risk_penalty * self._compute_rolling_vol(self.current_idx)**2
        dd = max(0, self.peak_equity - self.equity) / (self.peak_equity + 1e-6)
        dd_term = dd_penalty * dd
        turnover_term = turnover_penalty * trade_size

        reward = pnl - cost - risk_term - dd_term - turnover_term

        return reward

# Create environment
env = TradingEnvironment(market_data, CONFIG)

# Save environment spec
env_spec = {
    'version': 'v1.0_real_data',
    'state_variables': [
        'lagged_returns (20 lags)',
        'rolling_volatility (20-period)',
        'current_position',
        'equity',
        'drawdown',
        'cumulative_turnover'
    ],
    'state_dimension': env._get_state().shape[0],
    'action_space': {
        'type': 'continuous',
        'bounds': CONFIG['constraints']['position_bounds']
    },
    'reward_formula': 'pnl - cost - risk_penalty * vol^2 - dd_penalty * drawdown - turnover_penalty * turnover',
    'constraints': CONFIG['constraints'],
    'timing': {
        'decision': 't',
        'execution': 't+1',
        'reward_realization': 't+1'
    },
    'data_source': 'yfinance',
    'causality_guarantee': 'All features use data <= t only.'
}

env_spec_path = f"{PATHS['artifacts']}/environment_spec.json"
with open(env_spec_path, 'w') as f:
    json.dump(env_spec, f, indent=2)

print(f"[ENV] Environment created. State dim: {env_spec['state_dimension']}")

# Cell 7 — Baseline strategies
def run_episode(env, policy_fn, start_idx, end_idx):
    """Run a single episode with given policy."""
    state = env.reset(start_idx, end_idx)
    done = False

    trajectory = {
        'states': [],
        'actions': [],
        'rewards': [],
        'infos': []
    }

    while not done:
        action = policy_fn(state, env)
        trajectory['states'].append(state)
        trajectory['actions'].append(action)

        next_state, reward, done, info = env.step(action)
        trajectory['rewards'].append(reward)
        trajectory['infos'].append(info)

        state = next_state

    return trajectory

def cash_baseline(state, env):
    """Do-nothing baseline."""
    return 0.0

def buy_hold_baseline(state, env):
    """Buy-and-hold baseline."""
    return env.pos_max

def trend_baseline(state, env):
    """Trend following with vol targeting."""
    lagged_returns = state[:env.lookback]
    mean_return = np.mean(lagged_returns)

    vol = state[env.lookback]

    target_vol = 0.02
    if vol > 0:
        scale = target_vol / vol
    else:
        scale = 1.0

    if mean_return > 0:
        position = min(scale * env.pos_max, env.pos_max)
    elif mean_return < 0:
        position = max(-scale * env.pos_max, env.pos_min)
    else:
        position = 0.0

    return position

def myopic_baseline(state, env):
    """Myopic greedy baseline."""
    lagged_returns = state[:env.lookback]
    forecast = np.mean(lagged_returns)

    vol = state[env.lookback]
    if vol > 0:
        scale = 1.0 / vol
    else:
        scale = 1.0

    position = np.clip(forecast * scale * 10, env.pos_min, env.pos_max)

    return position

def compute_metrics(trajectory):
    """
    Compute metrics from trajectory.
    CORRECTED: Proper Sharpe ratio calculation.
    """
    rewards = np.array(trajectory['rewards'])
    infos = trajectory['infos']

    equity = np.array([info['equity'] for info in infos])

    net_return = equity[-1] - 1.0 if len(equity) > 0 else 0.0

    if len(equity) > 1:
        equity_returns = np.diff(equity) / (equity[:-1] + 1e-6)
        mean_return = np.mean(equity_returns)
        vol = np.std(equity_returns)

        # CORRECTED: Sharpe ratio calculation
        # Annualized Sharpe = (mean_return / vol) * sqrt(252)
        # For simplicity, use non-annualized: mean / std
        sharpe = (mean_return / vol) if vol > 0 else 0.0
    else:
        vol = 0.0
        sharpe = 0.0

    peak = np.maximum.accumulate(equity)
    drawdown = (peak - equity) / (peak + 1e-6)
    max_dd = np.max(drawdown) if len(drawdown) > 0 else 0.0

    turnover = sum(info['trade_size'] for info in infos)
    avg_cost = np.mean([info['cost'] for info in infos])

    violations = sum(1 for info in infos if info['constraint_violation'])
    violation_rate = violations / len(infos) if len(infos) > 0 else 0.0

    return {
        'net_return': net_return,
        'volatility': vol,
        'sharpe': sharpe,
        'max_drawdown': max_dd,
        'turnover': turnover,
        'avg_cost': avg_cost,
        'violation_rate': violation_rate,
        'total_steps': len(infos)
    }

# Run baselines
print("[BASELINES] Running baseline strategies...")

baseline_policies = {
    'cash': cash_baseline,
    'buy_hold': buy_hold_baseline,
    'trend': trend_baseline,
    'myopic': myopic_baseline
}

baseline_results = {}

eval_start = 50
eval_end = min(500, env.T - 50)

for name, policy_fn in baseline_policies.items():
    print(f"[BASELINES] Running {name}...")
    trajectory = run_episode(env, policy_fn, eval_start, eval_end)
    metrics = compute_metrics(trajectory)
    baseline_results[name] = metrics
    print(f"  Net return: {metrics['net_return']:.4f}, Sharpe: {metrics['sharpe']:.4f}, Max DD: {metrics['max_drawdown']:.4f}")

baseline_path = f"{PATHS['artifacts']}/baseline_metrics.json"
with open(baseline_path, 'w') as f:
    json.dump(baseline_results, f, indent=2)

# Plot baseline equity curves
plt.figure(figsize=(12, 6))
for name, policy_fn in baseline_policies.items():
    trajectory = run_episode(env, policy_fn, eval_start, eval_end)
    equity = [info['equity'] for info in trajectory['infos']]
    plt.plot(equity, label=name)

plt.title('Baseline Strategy Equity Curves')
plt.xlabel('Time')
plt.ylabel('Equity')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig(f"{PATHS['plots']}/baseline_equity.png", dpi=100)
plt.close()

print("[BASELINES] Baseline results saved.")

# Cell 8 — RL training
class LinearPolicy:
    """Simple linear policy for continuous actions."""

    def __init__(self, state_dim, pos_min, pos_max, seed=None):
        if seed is not None:
            np.random.seed(seed)
        self.W = np.random.randn(state_dim) * 0.01
        self.b = 0.0
        self.pos_min = pos_min
        self.pos_max = pos_max

    def predict(self, state):
        """Predict action for given state."""
        action = np.dot(self.W, state) + self.b
        return np.clip(action, self.pos_min, self.pos_max)

    def get_params(self):
        """Get policy parameters."""
        return {'W': self.W.copy(), 'b': self.b}

    def set_params(self, params):
        """Set policy parameters."""
        self.W = params['W'].copy()
        self.b = params['b']

def collect_expert_trajectories(env, expert_policy_fn, n_episodes, start_idx, end_idx, step=50):
    """Collect trajectories from expert policy."""
    trajectories = []

    for i in range(n_episodes):
        ep_start = start_idx + i * step
        ep_end = min(ep_start + step, end_idx)
        if ep_end - ep_start < 30:
            break

        traj = run_episode(env, expert_policy_fn, ep_start, ep_end)
        trajectories.append(traj)

    return trajectories

def train_behavior_cloning(env, expert_trajectories, config):
    """Train policy via behavior cloning."""
    print("[BC] Training behavior cloning policy...")

    states = []
    actions = []
    for traj in expert_trajectories:
        states.extend(traj['states'])
        actions.extend(traj['actions'])

    states = np.array(states)
    actions = np.array(actions)
    n_samples = len(states)

    print(f"[BC] Training on {n_samples} samples")

    state_dim = states.shape[1]
    policy = LinearPolicy(state_dim, env.pos_min, env.pos_max, seed=config['training']['seed_train'])

    epochs = config['training']['bc_epochs']
    lr = config['training']['bc_lr']
    batch_size = config['training']['bc_batch_size']

    losses = []

    for epoch in range(epochs):
        indices = np.random.permutation(n_samples)
        epoch_loss = 0.0
        n_batches = 0

        for i in range(0, n_samples, batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_states = states[batch_indices]
            batch_actions = actions[batch_indices]

            predictions = np.array([policy.predict(s) for s in batch_states])

            loss = np.mean((predictions - batch_actions)**2)
            epoch_loss += loss
            n_batches += 1

            errors = predictions - batch_actions
            grad_W = (2.0 / len(batch_states)) * np.sum([errors[j] * batch_states[j] for j in range(len(errors))], axis=0)
            grad_b = (2.0 / len(batch_states)) * np.sum(errors)

            policy.W -= lr * grad_W
            policy.b -= lr * grad_b

        avg_loss = epoch_loss / n_batches
        losses.append(avg_loss)

        if (epoch + 1) % 20 == 0:
            print(f"  Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")

    print("[BC] Behavior cloning complete.")

    return policy, losses

def conservative_policy_improvement(env, bc_policy, train_start, train_end, config):
    """Conservative policy improvement."""
    print("[CPI] Starting conservative policy improvement...")

    cpi_policy = LinearPolicy(bc_policy.W.shape[0], env.pos_min, env.pos_max)
    cpi_policy.set_params(bc_policy.get_params())

    cpi_steps = config['training']['cpi_steps']
    cpi_lr = config['training']['cpi_lr']
    deviation_penalty = config['training']['cpi_deviation_penalty']

    rewards_history = []

    eval_start = int(train_start)
    eval_end = int(train_end)

    for step in range(cpi_steps):
        traj = run_episode(env, lambda s, e: cpi_policy.predict(s), eval_start, eval_end)
        avg_reward = np.mean(traj['rewards'])
        rewards_history.append(avg_reward)

        epsilon = 0.01
        grad_W = np.zeros_like(cpi_policy.W)

        n_dims_sample = min(5, len(cpi_policy.W))
        sampled_dims = np.random.choice(len(cpi_policy.W), n_dims_sample, replace=False)

        for i in sampled_dims:
            cpi_policy.W[i] += epsilon
            traj_plus = run_episode(env, lambda s, e: cpi_policy.predict(s), eval_start, eval_end)
            reward_plus = np.mean(traj_plus['rewards'])

            cpi_policy.W[i] -= 2 * epsilon
            traj_minus = run_episode(env, lambda s, e: cpi_policy.predict(s), eval_start, eval_end)
            reward_minus = np.mean(traj_minus['rewards'])

            cpi_policy.W[i] += epsilon

            grad_W[i] = (reward_plus - reward_minus) / (2 * epsilon)

        deviation = cpi_policy.W - bc_policy.W
        grad_deviation = 2 * deviation_penalty * deviation

        cpi_policy.W += cpi_lr * (grad_W - grad_deviation)

        print(f"  CPI step {step+1}/{cpi_steps}, Avg reward: {avg_reward:.6f}")

    print("[CPI] Conservative policy improvement complete.")

    return cpi_policy, rewards_history

# Training
train_start = 50
train_end = min(400, env.T - 100)

print("[TRAIN] Collecting expert trajectories...")
expert_trajectories = collect_expert_trajectories(
    env, trend_baseline, n_episodes=6, start_idx=train_start, end_idx=train_end, step=50
)
print(f"[TRAIN] Collected {len(expert_trajectories)} expert trajectories")

bc_policy, bc_losses = train_behavior_cloning(env, expert_trajectories, CONFIG)

bc_policy_path = f"{PATHS['policy']}/bc_policy.npz"
np.savez(bc_policy_path, W=bc_policy.W, b=np.array([bc_policy.b]))
print(f"[TRAIN] BC policy saved")

cpi_policy, cpi_rewards = conservative_policy_improvement(env, bc_policy, train_start, train_end, CONFIG)

cpi_policy_path = f"{PATHS['policy']}/cpi_policy.npz"
np.savez(cpi_policy_path, W=cpi_policy.W, b=np.array([cpi_policy.b]))
print(f"[TRAIN] CPI policy saved")

training_traces = {
    'bc_losses': [float(x) for x in bc_losses],
    'cpi_rewards': [float(x) for x in cpi_rewards],
    'n_expert_trajectories': len(expert_trajectories),
    'total_expert_samples': sum(len(t['states']) for t in expert_trajectories)
}

traces_path = f"{PATHS['logs']}/training_traces.json"
with open(traces_path, 'w') as f:
    json.dump(training_traces, f, indent=2)

# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(bc_losses)
axes[0].set_title('Behavior Cloning Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(cpi_rewards)
axes[1].set_title('CPI Average Reward')
axes[1].set_xlabel('CPI Step')
axes[1].set_ylabel('Avg Reward')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{PATHS['plots']}/training_curves.png", dpi=100)
plt.close()

print("[TRAIN] Training complete.")

# Cell 9 — Walk-forward evaluation
def walk_forward_evaluation(env, policies, config):
    """Perform walk-forward evaluation."""
    print("[EVAL] Starting walk-forward evaluation...")

    eval_config = config['evaluation']
    train_len = eval_config['train_len']
    test_len = eval_config['test_len']
    step_len = eval_config['step_len']
    min_start = eval_config['min_train_start']

    T = env.T
    results = []

    fold = 0
    train_start = min_start

    while train_start + train_len + test_len < T:
        train_end = train_start + train_len
        test_start = train_end
        test_end = test_start + test_len

        print(f"[EVAL] Fold {fold}: train [{train_start}, {train_end}], test [{test_start}, {test_end}]")

        fold_results = {
            'fold': fold,
            'train_start': train_start,
            'train_end': train_end,
            'test_start': test_start,
            'test_end': test_end,
            'policies': {}
        }

        for name, policy_fn in policies.items():
            traj = run_episode(env, policy_fn, test_start, test_end)
            metrics = compute_metrics(traj)
            fold_results['policies'][name] = metrics
            print(f"  {name}: return={metrics['net_return']:.4f}, sharpe={metrics['sharpe']:.4f}")

        results.append(fold_results)

        train_start += step_len
        fold += 1

        if fold >= 3:
            break

    return results

eval_policies = {
    'cash': cash_baseline,
    'buy_hold': buy_hold_baseline,
    'trend': trend_baseline,
    'myopic': myopic_baseline,
    'bc_policy': lambda s, e: bc_policy.predict(s),
    'cpi_policy': lambda s, e: cpi_policy.predict(s)
}

wf_results = walk_forward_evaluation(env, eval_policies, CONFIG)

eval_path = f"{PATHS['artifacts']}/evaluation_suite.json"
with open(eval_path, 'w') as f:
    json.dump(wf_results, f, indent=2, default=float)

# Plot per-fold equity curves
n_folds = len(wf_results)
fig, axes = plt.subplots(n_folds, 1, figsize=(12, 4*n_folds))
if n_folds == 1:
    axes = [axes]

for i, fold_result in enumerate(wf_results):
    test_start = fold_result['test_start']
    test_end = fold_result['test_end']

    for name in ['trend', 'bc_policy', 'cpi_policy']:
        if name in eval_policies:
            traj = run_episode(env, eval_policies[name], test_start, test_end)
            equity = [info['equity'] for info in traj['infos']]
            axes[i].plot(equity, label=name)

    axes[i].set_title(f"Fold {i} Equity Curves")
    axes[i].set_ylabel('Equity')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

axes[-1].set_xlabel('Time')
plt.tight_layout()
plt.savefig(f"{PATHS['plots']}/walkforward_equity.png", dpi=100)
plt.close()

print("[EVAL] Walk-forward evaluation complete.")

# Cell 10 — Final summary
print("\n" + "="*80)
print(f"CHAPTER 19: RL FOR TRADING - REAL DATA ({market_data['ticker']}) - COMPLETE")
print("="*80)
print(f"Run ID: {run_id}")
print(f"Base path: {BASE_PATH}")
print(f"\nData: {market_data['ticker']}, {market_data['T']} days")
print(f"Period: {market_data['start_date']} to {market_data['end_date']}")

print("\n[CORRECTED] Sharpe Ratio Calculation:")
print("  Formula: (mean_return / std_return)")
print("  Expected range: -2 to +3 for most strategies")

if len(wf_results) > 0:
    print("\nWalk-Forward Results (Fold 0):")
    fold_0 = wf_results[0]
    for policy_name in ['trend', 'bc_policy', 'cpi_policy']:
        if policy_name in fold_0['policies']:
            metrics = fold_0['policies'][policy_name]
            print(f"  {policy_name}:")
            print(f"    Return: {metrics['net_return']:.4f}")
            print(f"    Sharpe: {metrics['sharpe']:.4f}")
            print(f"    Max DD: {metrics['max_drawdown']:.4f}")

print("\nAll artifacts saved to:", BASE_PATH)
print("="*80)

[SETUP] Installing yfinance...
[SETUP] All dependencies installed successfully.
[INIT] Run ID: 20251229_173330
[INIT] Base path: /content/ch19_runs/20251229_173330
[INIT] Hashing utilities ready.
[CONFIG] Config hash: 2b87b4d5060bb21843f9adaadbc2180effbcbd21c76d0fc1907ebb097daa8345
[CONFIG] Config and manifest saved.
[DATA] Downloading SPY from yfinance...
[DATA] Downloaded 1237 days of data
[DATA] Downloaded 1236 timesteps for SPY
[DATA] Returns fingerprint: 6754e3d644d810d5
[DATA] Market data plots saved.
[COSTS] Cost model registry saved.
[ENV] Environment created. State dim: 25
[BASELINES] Running baseline strategies...
[BASELINES] Running cash...
  Net return: 0.0000, Sharpe: 0.0000, Max DD: 0.0000
[BASELINES] Running buy_hold...
  Net return: 1.0409, Sharpe: 0.1335, Max DD: 0.0944
[BASELINES] Running trend...
  Net return: -0.1796, Sharpe: -0.0399, Max DD: 0.2675
[BASELINES] Running myopic...
  Net return: -0.3186, Sharpe: -0.0667, Max DD: 0.3594
[BASELINES] Baseline results save

##15.CONCLUSIONS


This chapter has taken you on a comprehensive journey through reinforcement learning for
trading decisions, moving deliberately from theoretical foundations to practical implementation
to governance-ready deployment. We began with a bold claim: RL offers a fundamentally different
approach to trading by optimizing sequential decisions rather than predicting individual outcomes.
Now, having built a complete system from scratch, we can assess what RL actually delivers—and
what it demands in return.

**What We Built and Why It Matters**

We constructed a minimal but complete RL trading system using only NumPy and Python's standard
library. This transparency was pedagogical by design—every algorithm, every calculation, every
design choice was explicit and inspectable. We implemented behavior cloning to learn from sensible
expert policies, conservative policy improvement to make cautious enhancements, and walk-forward
evaluation to test out-of-sample performance. We stress-tested under cost inflation, liquidity
shocks, and regime changes. We generated comprehensive governance artifacts: configuration
manifests, data fingerprints, environment specifications, decision logs, and reproducible bundles.

This wasn't just an academic exercise. The system we built embodies production-grade design
principles: deterministic reproducibility through seed management, causality enforcement through
explicit timing conventions, constraint satisfaction through projection operators, and cost
awareness through realistic transaction modeling. These aren't optional refinements—they're
survival requirements. A system that ignores any of these will fail catastrophically when deployed
with real capital, regardless of how impressive its backtested returns appear.

**The Performance Reality Check**

Our results on real market data (SPY from 2020-2024) revealed uncomfortable truths. In the first
walk-forward fold, the trend-following baseline lost 15% with a Sharpe ratio around -0.7. The RL
policies—both behavior cloning and conservative policy improvement—performed similarly poorly,
losing 2-3% with Sharpe ratios around -0.2 to -0.3. None of the strategies made money in this
out-of-sample period.

This is not a failure of the methodology—it's reality asserting itself. The 2020-2024 period
included extreme volatility (COVID crash), unprecedented monetary policy shifts, and regime changes
that our training data didn't adequately prepare us for. Our simple trend-following expert policy,
trained on one market regime, struggled when conditions changed. The RL policies learned to imitate
this struggling expert and made only marginal improvements through conservative policy optimization.

This outcome teaches a crucial lesson: RL is not magic. It cannot extract profits from markets
that offer no exploitable patterns to the strategies it has learned. If your expert policy (the
behavior cloning source) doesn't work out-of-sample, your RL policy won't either—it's learning
to do what the expert does, not to discover entirely new profitable strategies from scratch.

**What RL Actually Provides**

Given these sobering results, what value does RL offer for trading? The answer lies in what it
optimizes and how it learns.

First, RL naturally incorporates everything that matters for actual trading: transaction costs,
position constraints, risk penalties, and the delayed consequences of decisions. Traditional
supervised learning predicts returns; RL optimizes trading decisions accounting for the friction
and constraints of implementation. This cost-awareness is baked into the reward function, so the
agent automatically learns to avoid excessive turnover and to time trades when liquidity is better.

Second, RL provides a framework for systematic improvement. Behavior cloning captures institutional
knowledge (the expert's strategy), while conservative policy improvement makes data-driven
refinements. This two-stage approach is much safer than trying to learn optimal policies from
scratch in the dangerous extrapolation regime where offline RL typically fails.

Third, RL forces you to formalize your trading problem completely: What state information is
admissible? What actions are feasible? What outcomes do you actually care about? What constraints
must you respect? This formalization exercise alone—building the MDP specification—often reveals
unstated assumptions and hidden complexities that would otherwise cause silent failures in
production.

**The Offline RL Challenge**

Our demonstration of off-policy evaluation pitfalls crystallized a fundamental challenge: you
cannot reliably predict how a new policy will perform by analyzing data collected under a different
policy. Importance sampling weights explode when policies differ significantly, rendering estimates
useless. This is why we insisted on walk-forward backtests—actually running the policy in
simulated environments—rather than relying on clever statistical corrections.

This limitation shapes what's possible with offline RL for trading. You're constrained to learning
policies that don't deviate too far from the behavior policy that generated your historical data.
Conservative policy improvement respects this constraint by penalizing deviation, but it also
means your improvements are bounded. You can refine and polish existing strategies, but you
can't discover radically different approaches through offline learning alone.

Online RL—where the agent explores freely and learns from the consequences—could theoretically
overcome these limitations. But online learning in live markets is prohibitively expensive and
risky. You'd be experimenting with real capital, potentially losing substantial amounts while the
agent explores bad actions. For most trading applications, this is unacceptable.

**Governance as Competitive Advantage**

The governance artifacts we generated throughout this chapter—manifests, fingerprints, logs,
specifications, stress tests—might seem like bureaucratic overhead. They're not. They're the
difference between a research prototype and a deployable system.

When a strategy loses money (as ours did), governance artifacts let you diagnose why. You can
trace every decision back to the exact state that triggered it, verify that no future information
leaked into those states, confirm that constraints were enforced, and check whether costs were
calculated correctly. Without this traceability, you're left guessing whether the loss was bad
luck, a bug, or a fundamental flaw in your approach.

When regulators ask questions (and they will), governance artifacts provide answers. You can show
exactly what your system did, why it did it, what assumptions it made, and what risks you
considered. This documentation isn't about compliance theater—it's about demonstrating that you
operated responsibly and diligently.

When you want to improve the strategy, governance artifacts tell you where to focus. Maybe stress
tests reveal excessive cost sensitivity—improve execution. Maybe regime analysis shows the strategy
fails in high-volatility periods—add regime detection. Maybe decision logs show constraint
violations—tighten the projection operator. Artifacts transform vague intuitions into concrete
action items.

**Practical Recommendations for Practitioners**

If you're considering RL for trading in your organization, here are hard-won lessons from this
implementation:

**Start with strong baselines.** Your RL policy will only be as good as what it learns from. If
your expert policy (behavior cloning source) doesn't work out-of-sample, RL won't save you.
Invest in developing robust, cost-aware baselines before attempting RL.

**Be conservative with offline learning.** The conservative policy improvement approach we
demonstrated is not optional caution—it's essential for avoiding the extrapolation errors that
plague offline RL. Small, cautious steps away from demonstrated behavior are much safer than
aggressive optimization.

**Stress test relentlessly.** Our cost inflation and liquidity shock tests revealed that small
changes in assumptions can flip strategies from profitable to unprofitable. Test under 2-3×
normal costs, reduced liquidity, increased latency, and regime shifts. If your strategy only
works under ideal conditions, it doesn't work.

**Respect causality absolutely.** Every feature in your state must use only data available before
the decision. One forward-looking feature—a smoothed estimate, a future-peeking indicator—can
make a worthless strategy look profitable in backtests while guaranteeing losses in production.

**Invest in infrastructure.** The governance scaffolding we built—manifests, fingerprints, logs,
specifications—takes time to implement but pays dividends forever. It catches bugs during
development, enables debugging in production, and provides regulatory defense. Don't skip it.

**The Path Forward**

This chapter demonstrated RL for a single-asset, single-strategy problem with simple linear
policies. Real trading systems are vastly more complex: multiple assets, multiple strategies,
capital allocation across strategies, risk management overlays, regime detection, and portfolio
construction. Each layer adds complexity and new failure modes.

Future chapters will build on this foundation, but the principles remain constant: formalize your
problem as an MDP, learn conservatively from demonstrated behavior, evaluate rigorously out-of-
sample, stress test comprehensively, and document everything for governance. RL is a powerful
tool for sequential decision-making under constraints, but it's not a shortcut around the hard
work of strategy development, risk management, and operational excellence.

The negative returns we observed in our real-data evaluation are not the end of the story—they're
the beginning of the iterative improvement process. Now we know our simple trend-following approach
doesn't work on recent SPY data. Armed with decision logs, stress tests, and regime analysis, we
can diagnose why and hypothesize improvements: maybe we need regime-dependent policies, better
volatility forecasting, or different features in the state representation. Each iteration, informed
by governance artifacts, brings us closer to deployable performance.

Reinforcement learning for trading is not about finding a magic algorithm that prints money. It's
about building systematic, auditable, improvable decision-making systems that respect the realities
of market microstructure, transaction costs, and risk constraints. When done rigorously—with
transparency, conservatism, and comprehensive governance—RL provides a principled framework for
this challenge. The framework we've built in this chapter gives you the foundation to tackle it.